# ACCORD.csv

In [2]:
import os
import pandas as pd
import numpy as np
import pprint
from tqdm import tqdm

## Extract features from raw data

In [3]:
# get the list of names of the csv files in the ../data folder
csv_files = [f for f in os.listdir('../../../DataSets') if f.endswith('.csv')]
csv_files.sort()
for i in range(len(csv_files)):
    print(i+1, csv_files[i])

1 accord_key.csv
2 activitystatus.csv
3 bloodpressure.csv
4 concomitantmeds.csv
5 cvdoutcomes.csv
6 ecg.csv
7 eye.csv
8 f01_inclusionexclusionsummary.csv
9 f02_bptrialscreening.csv
10 f03_lipidtrialscreening.csv
11 f07_baselinehistoryphysicalexam.csv
12 f08_09_glycemiamanagement.csv
13 f10_glycemiamedicationslog.csv
14 f13_intensivebpmanagement.csv
15 f14_standardbpmanagement.csv
16 f15_bptrialmedicationslog.csv
17 f16_lipidmedicationsmanagement.csv
18 f19_healthutilitiesindex.csv
19 f22_costsubstudy.csv
20 f23_hrql.csv
21 f26_dietquestionnaire.csv
22 f29_champsphysicalactivity.csv
23 f34_intervalhistoryfollowup.csv
24 f36_annualfollowupphysicalexam.csv
25 f49_standingbloodpressure.csv
26 hba1c.csv
27 hypoglycemiaevents.csv
28 hypoglycemiatime1st.csv
29 lipids.csv
30 microvascularoutcomes.csv
31 mind.csv
32 mind_mri.csv
33 otherlabs.csv
34 sae.csv


In [4]:
# dfine the dict to select the columns from the csv files

red_dict = dict()
blue_dict = dict()

# the accord_key.csv file contains the basic information about the patients, no per visit data +++++
red_dict['accord_key.csv'] = ['MaskID', 'female', 'baseline_age','cvd_hx_baseline','raceclass']
blue_dict['accord_key.csv'] = ['MaskID', 'female', 'baseline_age','cvd_hx_baseline','raceclass']


# the following files contains the per visit data
red_dict['activitystatus.csv'] = ['MaskID', 'Visit', 'glycemia','bp']
blue_dict['activitystatus.csv'] = ['MaskID', 'Visit', 'glycemia','bp']

red_dict['bloodpressure.csv'] = ['MaskID', 'Visit', 'sbp', 'dbp', 'hr']
blue_dict['bloodpressure.csv'] = ['MaskID', 'Visit', 'sbp']

# no 'Visit' column in the file +++++
red_dict['cvdoutcomes.csv'] = ['MaskID', 'type_po']
blue_dict['cvdoutcomes.csv'] = ['MaskID', 'type_po']

red_dict['hba1c.csv'] = ['MaskID', 'Visit', 'hba1c']
blue_dict['hba1c.csv'] = ['MaskID', 'Visit', 'hba1c']

red_dict['lipids.csv'] = ['MaskID', 'Visit', 'chol', 'trig', 'vldl', 'ldl', 'hdl']
blue_dict['lipids.csv'] = ['MaskID', 'Visit', 'chol', 'hdl']

# no blue features +++++
red_dict['otherlabs.csv'] = ['MaskID', 'Visit', 'fpg', 'alt', 'cpk', 'potassium', 'screat', 'gfr', 'ualb','ucreat', 'uacr']
# blue_dict['otherlabs.csv'] = []

# only baseline visit info +++++
red_dict['f07_baselinehistoryphysicalexam.csv'] = ['MaskID', 'Visit', 'edu', 'yrsdiab', 'yrstens', 'cigarett', 'wt_kg', 'ht_cm']
blue_dict['f07_baselinehistoryphysicalexam.csv'] = ['MaskID', 'Visit', 'edu', 'yrsdiab', 'yrstens', 'cigarett', 'wt_kg', 'ht_cm']

# to calculate the BMI for every visit
red_dict['f36_annualfollowupphysicalexam.csv'] = ['MaskID', 'Visit', 'wt_kg', 'ht_cm']
blue_dict['f36_annualfollowupphysicalexam.csv'] = ['MaskID', 'Visit', 'wt_kg', 'ht_cm']

# multiple rows for the same MaskID and Visit, each row is a different medication +++++
red_dict['f10_glycemiamedicationslog.csv'] = ['MaskID', 'Visit', 'oral_gmed'] 
blue_dict['f10_glycemiamedicationslog.csv'] = ['MaskID', 'Visit', 'oral_gmed']

red_dict['f13_intensivebpmanagement.csv'] = ['MaskID', 'Visit', 'medadd','medchg']
blue_dict['f13_intensivebpmanagement.csv'] = ['MaskID', 'Visit', 'medadd','medchg']

red_dict['f14_standardbpmanagement.csv'] = ['MaskID', 'Visit', 'medchg']
blue_dict['f14_standardbpmanagement.csv'] = ['MaskID', 'Visit', 'medchg']

# multiple rows for the same MaskID and Visit, each row is a different medication +++++
red_dict['f15_bptrialmedicationslog.csv'] = ['MaskID', 'Visit','bp_med'] 
blue_dict['f15_bptrialmedicationslog.csv'] = ['MaskID', 'Visit','bp_med']

print('len(red_dict) = ', len(red_dict))
print('len(blue_dict) = ', len(blue_dict))

# count total number of features in the red and blue dictionaries
red_features_set = set()
blue_features_set = set()
for key in red_dict:
    red_features_set.update(red_dict[key])
for key in blue_dict:
    blue_features_set.update(blue_dict[key])

print('red_features = ', len(red_features_set)+1) # add 1 bc duplicate medchg
print('blue_features = ', len(blue_features_set)+1)

len(red_dict) =  13
len(blue_dict) =  12
red_features =  38
blue_features =  24


In [7]:
# use the red_dict and blue_dict to create the red and blue datasets
# load the each csv file using the key, 
# and select the corresponding columns using the value
# and save the resulting dataframe to a csv file

def collect_feature_columnns(dict, flag):
    print('\ncollect_feature_columnns:', flag)

    min_row_number = 1e6
    for i, key in enumerate(dict):
        
        df = pd.read_csv('../../../DataSets/' + key)
        print('collect', i, key, ', row_number =', df.shape[0])
        if df.shape[0] < min_row_number:
            min_row_number = df.shape[0]

        # check if desired columns are in the dataframe
        for col in dict[key]:
            if col not in df.columns:
                print('Error in dict:', col, 'not in', key)

        df_fea = df[dict[key]]
        out_file = 'data/features_'+ str(flag)+'/' + key
        df_fea.to_csv(out_file, index=False)
        # print('save to', out_file)

    print('done')
    print('min_row_number =', min_row_number)    

collect_feature_columnns(red_dict, 'red')
# collect_feature_columnns(blue_dict, 'blue')


collect_feature_columnns: red
collect 0 accord_key.csv , row_number = 10251
collect 1 activitystatus.csv , row_number = 388624
collect 2 bloodpressure.csv , row_number = 181991
collect 3 cvdoutcomes.csv , row_number = 10251
collect 4 hba1c.csv , row_number = 136650
collect 5 lipids.csv , row_number = 69062
collect 6 otherlabs.csv , row_number = 118662
collect 7 f07_baselinehistoryphysicalexam.csv , row_number = 10251
collect 8 f36_annualfollowupphysicalexam.csv , row_number = 50701
collect 9 f10_glycemiamedicationslog.csv , row_number = 727628
collect 10 f13_intensivebpmanagement.csv , row_number = 68732
collect 11 f14_standardbpmanagement.csv , row_number = 40026
collect 12 f15_bptrialmedicationslog.csv , row_number = 307222
done
min_row_number = 10251


Total patients number: `10251`, as shown by the row number of `accord_key.csv` and `cvdoutcomes.csv` 

## Collect medication list per patient's visit

Process the files with medication prescription information `f10_glycemiamedicationslog.csv`--> `oral_gmed`, and `f15_bptrialmedicationslog.csv` --> `bp_med`.
Collect multiple rows with the same `MaskID` and `Visit` but different medications into the same row.
The medicine is converted to lowercase and sorted.

In [8]:
def collect_med(fn, col):
    print('\ncollect_med:', fn, col)

    df = pd.read_csv(fn)
    print(df.shape)
    # print(df.info())

    # for all rows with missing `col` values, replace with 'NAN'
    df[col] = df[col].fillna('none')

    # cast the `col` column to string
    # df[col] = df[col].astype('string')
    # print(df.info())

    # group by MaskID and Visit, and convert the `col` column to a list
    df = df.groupby(['MaskID', 'Visit'])[col].apply(list).reset_index()

    # convert to lowercase, then sort the list in the `col` column for each row
    df[col] = df[col].apply(lambda x: sorted([y.lower() for y in x]))
     
    df[col] = df[col].apply(lambda x: '+'.join(sorted(x)))
    # print(df.info())
    print(df.shape)

    # print(df.tail())
    outfn = fn.replace('.csv', '_medlist.csv')
    df.to_csv(outfn, index=False)
    print('save to', outfn)

collect_med('data/features_red/f10_glycemiamedicationslog.csv', 'oral_gmed')
# collect_med('data/features_blue/f10_glycemiamedicationslog.csv', 'oral_gmed')

collect_med('data/features_red/f15_bptrialmedicationslog.csv', 'bp_med')
# collect_med('data/features_blue/f15_bptrialmedicationslog.csv', 'bp_med')


collect_med: data/features_red/f10_glycemiamedicationslog.csv oral_gmed
(727628, 3)
(311310, 3)
save to data/features_red/f10_glycemiamedicationslog_medlist.csv

collect_med: data/features_red/f15_bptrialmedicationslog.csv bp_med
(307222, 3)
(104571, 3)
save to data/features_red/f15_bptrialmedicationslog_medlist.csv


## Modifiy duplicated column/feature names

* `f13_intensivebpmanagement.csv` and `f14_standardbpmanagement.csv` has the same column name `medchg`, we need to change it to `medchg_intbp` and `medchg_stdbp`
* `f07_baselinehistoryphysicalexam.csv`: `wt_kg` --> `wt_kg_baseline`, `ht_cm` --> `ht_cm_baseline`. Also add '_baseline' to other features in this file.
* `f36_annualfollowupphysicalexam.csv`: `wt_kg` --> `wt_kg_visit`, `ht_cm` --> `ht_cm_visit`

In [9]:
def modify_dup_name(fn, old_col_name_list, new_col_name_list, do_blue=False):
    fn1 = 'data/features_red/' + fn
    

    # build a dictionary to rename the columns
    name_pairs_dict = {}
    for i in range(len(old_col_name_list)):
        name_pairs_dict[old_col_name_list[i]] = new_col_name_list[i]

    df1 = pd.read_csv(fn1)
    df1.rename(columns=name_pairs_dict, inplace=True)
    df1.to_csv(fn1, index=False)
    print('change', old_col_name_list, 'to', new_col_name_list, 'in', fn1)
    print('save to', fn1)

    if do_blue:
        fn2 = 'data/features_blue/' + fn
        df2 = pd.read_csv(fn2)
        df2.rename(columns=name_pairs_dict, inplace=True)
        df2.to_csv(fn2, index=False)
        print('change', old_col_name_list, 'to', new_col_name_list, 'in', fn2)
        print('save to', fn2)
        print()

modify_dup_name('f13_intensivebpmanagement.csv', ['medchg'], ['medchg_intbp'])
modify_dup_name('f14_standardbpmanagement.csv', ['medchg'], ['medchg_stdbp'])
modify_dup_name('f07_baselinehistoryphysicalexam.csv', ['edu', 'yrsdiab', 'yrstens', 'cigarett','wt_kg','ht_cm'], 
                                                       ['edu_baseline','yrsdiab_baseline','yrstens_baseline','cigarett_baseline','wt_kg_baseline','ht_cm_baseline'])
modify_dup_name('f36_annualfollowupphysicalexam.csv', ['wt_kg','ht_cm'], ['wt_kg_visit','ht_cm_visit'])

change ['medchg'] to ['medchg_intbp'] in data/features_red/f13_intensivebpmanagement.csv
save to data/features_red/f13_intensivebpmanagement.csv
change ['medchg'] to ['medchg_stdbp'] in data/features_red/f14_standardbpmanagement.csv
save to data/features_red/f14_standardbpmanagement.csv
change ['edu', 'yrsdiab', 'yrstens', 'cigarett', 'wt_kg', 'ht_cm'] to ['edu_baseline', 'yrsdiab_baseline', 'yrstens_baseline', 'cigarett_baseline', 'wt_kg_baseline', 'ht_cm_baseline'] in data/features_red/f07_baselinehistoryphysicalexam.csv
save to data/features_red/f07_baselinehistoryphysicalexam.csv
change ['wt_kg', 'ht_cm'] to ['wt_kg_visit', 'ht_cm_visit'] in data/features_red/f36_annualfollowupphysicalexam.csv
save to data/features_red/f36_annualfollowupphysicalexam.csv


## Merge tables

1. The row numbers of each csv files are shown above. If we just naively merge them using `MaskID` and `Visit` column values, to keep the matching rows only, the resulting table would only contains `1725` rows, which looses too many datapoints.
2. So, I use `how='outer'` when merging. The resulting table would have `at least 388624` rows (since the `activitystatus.csv` has the most number of rows, 388624 rows), with many missing entries. The table would contain all visits for all patients.

In [11]:
# merge a list of dataframes into a single dataframe, using the common column 'MaskID' and 'Visit'

def merge_dfs(dfs, common_col):
    df = dfs[0]
    for i in range(1, len(dfs)):
        # df = df.merge(dfs[i], on=common_col) # default is 'inner' join
        df = df.merge(dfs[i], on=common_col, how='outer')
    return df

# merge the red/blue features
def combine_features(dict, flag):
    print('\ncombine_features:', flag)

    dfs = []
    for fn in dict:

        if fn == 'accord_key.csv' or fn == 'cvdoutcomes.csv': # this two files have no 'Visit' column, will add later on
            continue

        # update feature file names, use the medication list files
        if fn == 'f10_glycemiamedicationslog.csv':
            fn = 'f10_glycemiamedicationslog_medlist.csv'
        if fn == 'f15_bptrialmedicationslog.csv':
            fn = 'f15_bptrialmedicationslog_medlist.csv'

        folder = 'data/features_' + str(flag) + '/'
        df= pd.read_csv(folder + fn)
        dfs.append(df)

    df_res = merge_dfs(dfs, ['MaskID', 'Visit'])
    print('df_res.shape =', df_res.shape)

    # set the order for the columns

    # sort the rows by MaskID first then by Visit to ensure the visit are in order and complete for a patient
    # change the Visit column with 'EXIT' value to 'X-EXIT'
    df_res['Visit'] = df_res['Visit'].apply(lambda x: 'X-EXIT' if x == 'EXIT' else x)
    df_res.sort_values(by=['MaskID', 'Visit'], inplace=True)


    # save the merged dataframe to a csv file
    df_res.to_csv('data/temp/features_' + str(flag) + '_merge.csv', index=False)

combine_features(red_dict, 'red')
# combine_features(blue_dict, 'blue')


combine_features: red
df_res.shape = (388771, 35)


The red and blue datasets have different row number (388771 vs. 388679) because the blue dataset does not select features from the `otherlabs.csv`.

In [12]:
# check if have duplicated rows with same MaskID and Visit
df_red = pd.read_csv('data/temp/features_red_merge.csv')
print('df_red.shape =', df_red.shape)
df_red = df_red.drop_duplicates(subset=['MaskID', 'Visit'])
print('df_red.shape =', df_red.shape)

# df_blue = pd.read_csv('data/temp/features_blue_merge.csv')
# print('df_blue.shape =', df_blue.shape)
# df_blue = df_blue.drop_duplicates(subset=['MaskID', 'Visit'])
# print('df_blue.shape =', df_blue.shape)

df_red.shape = (388771, 35)
df_red.shape = (388771, 35)


In [ ]:
As shown above, the row numbers does not change, thus no duplicated rows.

In [13]:
# show the resulting table
print(df_red.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 388771 entries, 0 to 388770
Data columns (total 35 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   MaskID             388771 non-null  int64  
 1   Visit              388771 non-null  object 
 2   glycemia           375659 non-null  float64
 3   bp                 123919 non-null  float64
 4   sbp                181991 non-null  float64
 5   dbp                181969 non-null  float64
 6   hr                 181752 non-null  float64
 7   hba1c              136650 non-null  float64
 8   chol               69061 non-null   float64
 9   trig               69061 non-null   float64
 10  vldl               69051 non-null   float64
 11  ldl                69051 non-null   float64
 12  hdl                69053 non-null   float64
 13  fpg                79017 non-null   float64
 14  alt                87037 non-null   float64
 15  cpk                58217 non-null   float64
 16  po

In [ ]:
# print(df_blue.info())

## Filter out inactive patients in both BP and BG trials

Now the datasets have no duplicated patient-visit rows, let's do further processing

1. Filtering out rows with inactive status in both glycemia and bp status

Notice that, from observation of the data, a patient may: 
* quit early in BP/BG trials
* quit in the middle of visit, and join back in later visit, e.g. MaskID 102990
* But if they have ever been active, their baseline visit (BLR) must be active

Thus, I use BLR visit to filter out patients who neither participate in BP nor BG trials.

In [14]:
# use the glycemia and bp at BLR visit (baseline) †o select the candidates
def select_active_blr(fn):

    print('fn =', fn)
    df = pd.read_csv(fn)
    print('original df.shape =', df.shape)

    # select the rows with visit = BLR, confirmed every patient has only one row with visit = BLR, total 10251 patients
    df_blr = df[df['Visit'] == 'BLR']
    print('select only BLR rows, df_blr.shape =', df_blr.shape)

    df_exit = df[df['Visit'] == 'X-EXIT']
    print('select only EXIT rows, df_exit.shape =', df_exit.shape) # only 9146 rows with visit = EXIT, some have missing EXIT visit

    # select the rows with glycemia = 1
    df_bg = df_blr[df_blr['glycemia'] == 1]
    print('active glycemia, df_bg.shape =', df_bg.shape)

    # select the rows with bp = 1
    df_bp = df_blr[df_blr['bp'] == 1]
    print('actve bp, df_bp.shape =', df_bp.shape) # remaining 4733 patients with glycemia = 1 and bp = 1
    
    # collect the MaskID of the selected patients into a set
    candidates_bg = set(df_bg['MaskID'])
    candidates_bp = set(df_bp['MaskID'])
    candidates = candidates_bg.union(candidates_bp)
    print('candidates number with either active BP or BG =', len(candidates))

    # save the candidates_bg and candidates_bp to a csv file
    candidates_bg_lst = list(candidates_bg)
    candidates_bp_lst = list(candidates_bp)

    # write the candidates_bg_lst to a file
    with open('data/temp/candidates_bg.csv', 'w') as f:
        for item in candidates_bg_lst:
            f.write("%s\n" % item)
    
    # write the candidates_bp_lst to a file
    with open('data/temp/candidates_bp.csv', 'w') as f:
        for item in candidates_bp_lst:
            f.write("%s\n" % item)


    #------- select the rows in the original table with MaskID in the set
    # df1 = pd.read_csv(fn)
    # print('original df1.shape =', df1.shape)
    # df1 = df1[df1['MaskID'].isin(candidates)]
    # print('select candidates using BLR, df1.shape =', df1.shape)

    #------- save the selected rows to a csv file
    # df1.to_csv('data/temp/features_' + str(flag) + '_merge_blr.csv', index=False)  

select_active_blr('data/temp/features_red_merge.csv')
# select_active_blr('data/temp/features_blue_merge.csv')

fn = data/temp/features_red_merge.csv
original df.shape = (388771, 35)
select only BLR rows, df_blr.shape = (10251, 35)
select only EXIT rows, df_exit.shape = (9146, 35)
active glycemia, df_bg.shape = (10251, 35)
actve bp, df_bp.shape = (4733, 35)
candidates number with either active BP or BG = 10251


Results above shows that all 10251 patients in record have active BG status, and 4733 of them have active BP. Thus we keep all of the patient-visit rows.

## Calculate BMI per visit

1. put the value of columns 'wt_kg_baseline' and 'ht_cm_baseline' to the columns 'ht_cm_visit' and 'wt_kg_visit'
2. fill in the missing values in the columns 'ht_cm_visit' and 'wt_kg_visit' with -1, DONOT take the average of the previous and next valid values!
3. calculate the BMI per visit, for missing wt or ht, mark as -1

In [15]:
def fill_with_adjacent_avg_list(old_list, avg_value):
  
    new_list = old_list.copy()

    # build a dictionary to store the index, value pairs in old_list
    old_dict = {}
    for i in range(len(old_list)):        
        if old_list[i] == -1: # skip the missing values, +++++ notice here, the missing values are marked with -1
            continue
        else:
            old_dict[i] = old_list[i] 
        
    old_dict_keys = list(old_dict.keys())
    # print('old_dict_keys =', old_dict_keys)
    # print('old_dict =', old_dict)

    if (len(old_dict_keys) == len(old_list)): # no missing values
        return new_list

    # if all values are missing, fill with the average value
    if len(old_dict) == 0:
        new_list = [avg_value] * len(old_list)

    # if there is only one value, fill the entire list with the value
    elif len(old_dict) == 1:
        first_key = old_dict_keys[0]
        new_list = [old_dict[first_key]] * len(old_list)
    
    # if there are more than one values, fill the missing values with the average value of the previous and next values
    else: #len(old_dict) > 1:
        
        # fill in all missing values before the first valid value using the first valid value
        first_key = old_dict_keys[0]
        first_value = old_dict[first_key]
        first_index = first_key
        for i in range(first_index):
            new_list[i] = first_value
        
        # fill in all missing values after the last valid value using the last valid value
        last_key = old_dict_keys[-1]
        last_value = old_dict[last_key]
        last_index = last_key
        for i in range(last_index+1, len(old_list)):
            new_list[i] = last_value
        
        # for the remaining index missing values, fill with the average value of the its previous and next values
        prev_value = first_value
        old_dict_keys_idx = 0 #
        i = first_index+1
        while i < last_index:

            if i in old_dict: # valid value, update the prev_value
                prev_value = old_dict[i]
                old_dict_keys_idx += 1 # update the index of the old_dict_keys
                i += 1
                continue
            else:
                # find the next_value by moving to the next index in old_dict_keys
                next_index = old_dict_keys[old_dict_keys_idx+1]
                next_value = old_dict[next_index]

                for j in range(i, next_index):
                    new_list[j] = float((prev_value + next_value) / 2)
                
                # move the i to the next_index+1
                old_dict_keys_idx += 1 # update the index of the old_dict_keys
                i = next_index+1
                prev_value = next_value

    # print('old_list =', old_list)
    # print('new_list =', new_list)

    return new_list

def fill_with_adjacent_avg(df, col_name, avg_value):

    new_col = []
    # select each patient's rows by MaskID
    # for maskid in tqdm(range(100001, 110252)):
    for maskid in range(100001, 110252):
        # print('maskid =', maskid)
        df_patient = df[df['MaskID'] == maskid]
        patient_col = df_patient[col_name].tolist()
        # print('patient_col =', patient_col)

        # fill the missing values with the average value of the previous and next visits
        new_patient_col = fill_with_adjacent_avg_list(patient_col, avg_value)
        
        new_col.extend(new_patient_col)
  
    assert(len(new_col) == df.shape[0])
    df[col_name] = new_col

    return df


def add_BMI(fn):
    df = pd.read_csv(fn)
    
    # for rows with 'Visit'=BLR, put the value of columns 'wt_kg_baseline' and 'ht_cm_baseline' 
    # to the columns 'ht_cm_visit' and 'wt_kg_visit'
    for i in range(df.shape[0]):
        if df.loc[i, 'Visit'] == 'BLR':
            df.loc[i, 'wt_kg_visit'] = df.loc[i, 'wt_kg_baseline']
            df.loc[i, 'ht_cm_visit'] = df.loc[i, 'ht_cm_baseline']
    
    # fill in the missing values of 'wt_kg_visit' and 'ht_cm_visit' with the average value of the previous and next visits
    # if the patient have no wt_kg_visit or ht_cm_visit value, use the average value of the whole population

    # calculate the average value of the whole population, donot account for missing values
    wt_kg_mean = df['wt_kg_visit'].mean()
    ht_cm_mean = df['ht_cm_visit'].mean()
    print('wt_kg_mean =', wt_kg_mean)
    print('ht_cm_mean =', ht_cm_mean)

    # fill in the missing values of 'wt_kg_visit' and 'ht_cm_visit' with -1
    df['wt_kg_visit'] = df['wt_kg_visit'].fillna(-1)
    df['ht_cm_visit'] = df['ht_cm_visit'].fillna(-1)

    # loop through all rows
    # df = fill_with_adjacent_avg(df, 'wt_kg_visit', wt_kg_mean)
    # df = fill_with_adjacent_avg(df, 'ht_cm_visit', ht_cm_mean)

    # calculate the BMI per visit
    BMI = []
    for i in range(df.shape[0]):
        row = df.loc[i]
        if row['wt_kg_visit'] == -1 or row['ht_cm_visit'] == -1:
            BMI.append(-1)
        else:
            BMI.append(df.loc[i, 'wt_kg_visit'] / (df.loc[i, 'ht_cm_visit']/100)**2)
            
    df['BMI'] = BMI # the BMI is per visit

    print(df.info())

    # save the dataframe to a csv file
    df.to_csv(fn.split('.')[0]+'_BMI.csv', index=False)

add_BMI('data/temp/features_red_merge.csv')
# add_BMI('data/temp/features_blue_merge.csv')

wt_kg_mean = 94.64666678052369
ht_cm_mean = 169.87821139400054
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 388771 entries, 0 to 388770
Data columns (total 36 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   MaskID             388771 non-null  int64  
 1   Visit              388771 non-null  object 
 2   glycemia           375659 non-null  float64
 3   bp                 123919 non-null  float64
 4   sbp                181991 non-null  float64
 5   dbp                181969 non-null  float64
 6   hr                 181752 non-null  float64
 7   hba1c              136650 non-null  float64
 8   chol               69061 non-null   float64
 9   trig               69061 non-null   float64
 10  vldl               69051 non-null   float64
 11  ldl                69051 non-null   float64
 12  hdl                69053 non-null   float64
 13  fpg                79017 non-null   float64
 14  alt                87037 non-null   f

In [17]:
# count how many rows have -1 for BMI
df = pd.read_csv('data/temp/features_red_merge_BMI.csv')
print(df[df['BMI'] != -1].shape[0])

58871


## Add the patient's info

1. Add the patient's information on from the following files to each row, using the `MaskID`
  * `accord_key.csv` 
  * `cvdoutcomes.csv` , only `1046` patients have valid `type_po` entries
  * `f07_baselinehistoryphysicalexam.csv`

In [18]:
# read in accord_key.csv and build a dict of dict
# the first dict is keyed by MaskID, the second dict is keyed by column names
def build_dict(fn1, fn2, fn3):
    print('\nbuild_dict:', fn1, fn2, fn3)
    
    df1 = pd.read_csv(fn1)
    df2 = pd.read_csv(fn2)
    df3 = pd.read_csv(fn3)
    print('df1.shape =', df1.shape)
    print('df2.shape =', df2.shape)
    print('df3.shape =', df3.shape)
    
    dict = {}
    # accord_key.csv
    for i in range(df1.shape[0]):
        row = df1.iloc[i]
        MaskID = row['MaskID']
        if MaskID not in dict:
            dict[MaskID] = {}
        else:
            print('error: duplicated MaskID =', MaskID)
            exit()

        for col in df1.columns:
            dict[MaskID][col] = row[col]
    
    # cvdoutcomes.csv
    for i in range(df2.shape[0]):
        row = df2.iloc[i]
        MaskID = row['MaskID']
        if MaskID not in dict:
            dict[MaskID] = {}

        for col in df2.columns:
            dict[MaskID][col] = row[col]

    # f07_baselinehistoryphysicalexam.csv
    for i in range(df3.shape[0]):
        row = df3.iloc[i]
        MaskID = row['MaskID']
        if MaskID not in dict:
            dict[MaskID] = {}

        for col in df3.columns:
            dict[MaskID][col] = row[col]    

    print('dict size =', len(dict))
    print('dict[100001] =', dict[100001])
    print('dict[110251] =', dict[110251])

    return dict

col_dict = build_dict('data/features_red/accord_key.csv', 
                      'data/features_red/cvdoutcomes.csv',
                      'data/features_red/f07_baselinehistoryphysicalexam.csv')


build_dict: data/features_red/accord_key.csv data/features_red/cvdoutcomes.csv data/features_red/f07_baselinehistoryphysicalexam.csv
df1.shape = (10251, 5)
df2.shape = (10251, 2)
df3.shape = (10251, 8)
dict size = 10251
dict[100001] = {'MaskID': 100001, 'female': 0, 'baseline_age': 60.8, 'cvd_hx_baseline': 0, 'raceclass': 'White', 'type_po': nan, 'Visit': 'BLR', 'edu_baseline': 4.0, 'yrsdiab_baseline': 26.0, 'yrstens_baseline': 6.0, 'cigarett_baseline': 2, 'wt_kg_baseline': 101.599998474121, 'ht_cm_baseline': 168.199996948242}
dict[110251] = {'MaskID': 110251, 'female': 0, 'baseline_age': 61.3, 'cvd_hx_baseline': 0, 'raceclass': 'Black', 'type_po': nan, 'Visit': 'BLR', 'edu_baseline': 4.0, 'yrsdiab_baseline': 10.0, 'yrstens_baseline': 2.0, 'cigarett_baseline': 2, 'wt_kg_baseline': 108.099998474121, 'ht_cm_baseline': 177.800003051758}


In [19]:
# add the features in accord_key.csv and cvdoutcomes.csv to the table
def add_fea_cols(fn, dict):
    
    print('\nadd_fea_cols:', fn)
    df = pd.read_csv(fn)

    # get feature names in dict
    first_key = list(dict.keys())[0]
    feature_names = list(dict[first_key].keys())
    print('feature_names =', feature_names)

    fea_dict = {}
    for fea in feature_names:
        fea_dict[fea] = []

    # add the values from the dict to the table row by row using matching MaskID
    for i in range(df.shape[0]):
        row = df.iloc[i]
        MaskID = row['MaskID']
        if MaskID in dict:
            for fea in feature_names:
                fea_dict[fea].append(dict[MaskID][fea])
        else:
            print('error: MaskID =', MaskID, 'not found')
            exit()
    
    # add the new columns to the table
    for fea in feature_names:
        if fea == 'MaskID' or fea == 'Visit': # donot add MaskID, sicne it is already in the table
            continue
        df[fea] = fea_dict[fea]

    print(df.info())
    
    df.to_csv(fn.split('.')[0]+'_addfea.csv', index=False)

add_fea_cols('data/temp/features_red_merge_BMI.csv', col_dict)
# add_fea_cols('data/temp/features_blue_merge_BMI.csv', col_dict)


add_fea_cols: data/temp/features_red_merge_BMI.csv
feature_names = ['MaskID', 'female', 'baseline_age', 'cvd_hx_baseline', 'raceclass', 'type_po', 'Visit', 'edu_baseline', 'yrsdiab_baseline', 'yrstens_baseline', 'cigarett_baseline', 'wt_kg_baseline', 'ht_cm_baseline']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 388771 entries, 0 to 388770
Data columns (total 41 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   MaskID             388771 non-null  int64  
 1   Visit              388771 non-null  object 
 2   glycemia           375659 non-null  float64
 3   bp                 123919 non-null  float64
 4   sbp                181991 non-null  float64
 5   dbp                181969 non-null  float64
 6   hr                 181752 non-null  float64
 7   hba1c              136650 non-null  float64
 8   chol               69061 non-null   float64
 9   trig               69061 non-null   float64
 10  vldl               69051 n

## Handle missing values
 
1. `type_po`: 0 means nothing, 1 means have a valid entry of CVD death, MI, Stroke
2. `medchg_intbp`, `medchg_stdbp`, `medadd`: 0 means No, 1 means Yes
3. `oral_gmed`, `bp_med`: fill in missing values with 'none'
4. yrstens_baseline, yrsdiab_baseline, edu_baseline: fill in missing values with -1
5. Other numeric entires, fille with average of adjacent values. For patients missing entire visit records, marked as -1

In [20]:
def handle_missing_values(fn, flag):
    print('\nhandle_missing_values:', fn)
    df = pd.read_csv(fn)
    # print(df.info())

    # fill in the missing values in 'type_po' with 0, and convert the valid values to 1
    df['type_po'] = df['type_po'].fillna('None')
    new_type_po = []   
    for i in range(df.shape[0]):
        if df.loc[i, 'type_po'] == 'None':
            new_type_po.append(0)
        else:
            new_type_po.append(1)
    df['type_po'] = new_type_po
    
    # 'medchg_intbp', 'medchg_stdbp', 'medadd'
    # fill in the missing values with 0
    df['medchg_intbp'] = df['medchg_intbp'].fillna(0)
    df['medchg_stdbp'] = df['medchg_stdbp'].fillna(0)
    df['medadd'] = df['medadd'].fillna(0)
     
    # bpmed, oral_gmed
    df['bp_med'] = df['bp_med'].fillna('none')
    df['oral_gmed'] = df['oral_gmed'].fillna('none')

    # wt_kg_baseline, ht_cm_baseline fill in missing values with -1 to show it is missing
    df['wt_kg_baseline'] = df['wt_kg_baseline'].fillna(-1)
    df['ht_cm_baseline'] = df['ht_cm_baseline'].fillna(-1)

    # yrstens_baseline, yrsdiab_baseline, edu_baseline
    # fill in missing values with -1 to show it is missing
    df['yrstens_baseline'] = df['yrstens_baseline'].fillna(-1)
    df['yrsdiab_baseline'] = df['yrsdiab_baseline'].fillna(-1)
    df['edu_baseline'] = df['edu_baseline'].fillna(-1)
    
    # change the column name 'chol' to 'TC'
    df.rename(columns={'chol':'TC'}, inplace=True)

    # other features based on flag
    if flag == 'red':
        col_names = ['sbp','dbp','hr', 'hba1c', 'TC', 'trig', 'vldl', 'ldl', 'hdl', 
                     'fpg', 'alt', 'cpk', 'potassium','screat', 'gfr', 'ualb', 'ucreat','uacr']

    elif flag == 'blue':
        col_names = ['sbp', 'hba1c', 'TC', 'hdl']
    else:
        print('error: invalid flag =', flag)
        exit()
    # print(df.columns)
    
    for col in tqdm(col_names):
        df[col] = df[col].fillna(-1) # fill in missing values with -1 to mark it is missing, then fill in with average of adjacent values
        # df = fill_with_adjacent_avg(df, col, -1) # note here, for a patient with no visit record, we fill in -1 marking as missing
    
    print(df.info())

    df.to_csv(fn.split('.')[0]+'_handlemissing.csv', index=False)

handle_missing_values('data/temp/features_red_merge_BMI_addfea.csv','red')
# handle_missing_values('data/temp/features_blue_merge_BMI_addfea.csv', 'blue')


handle_missing_values: data/temp/features_red_merge_BMI_addfea.csv


100%|██████████| 18/18 [00:00<00:00, 38.59it/s]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 388771 entries, 0 to 388770
Data columns (total 41 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   MaskID             388771 non-null  int64  
 1   Visit              388771 non-null  object 
 2   glycemia           375659 non-null  float64
 3   bp                 123919 non-null  float64
 4   sbp                388771 non-null  float64
 5   dbp                388771 non-null  float64
 6   hr                 388771 non-null  float64
 7   hba1c              388771 non-null  float64
 8   TC                 388771 non-null  float64
 9   trig               388771 non-null  float64
 10  vldl               388771 non-null  float64
 11  ldl                388771 non-null  float64
 12  hdl                388771 non-null  float64
 13  fpg                388771 non-null  float64
 14  alt                388771 non-null  float64
 15  cpk                388771 non-null  float64
 16  po

In [21]:
# count how many rows have value -1 in each column
def count_missing_values(fn):
    print('\ncount_missing_values:', fn)
    df = pd.read_csv(fn)
    # print(df.info())
    MaskID_to_remove = set()
    target_col = {'sbp', 'hdl', 'TC', 'hba1c'}
    print('columnname', 'missing_row_count', 'missing_MaskID_count')
    for col in df.columns:
        # count how many different MaskID for the rows with value -1 in the column
        MaskID_unique = df[df[col] == -1]['MaskID'].unique()
        print(col, df[df[col] == -1].shape[0], len(MaskID_unique))
        # add the MaskID to the set
        # print('col =', col, 'MaskID_unique =', MaskID_unique, 'MaskID_to_remove =', MaskID_to_remove)
        if col in target_col:
            for MaskID in MaskID_unique:
                MaskID_to_remove.add(MaskID)
    print()

    return MaskID_to_remove

MaskID_to_remove = count_missing_values('data/temp/features_red_merge_BMI_addfea_handlemissing.csv')
print('MaskID_to_remove =', MaskID_to_remove)
print('len(MaskID_to_remove) =', len(MaskID_to_remove))
# count_missing_values('data/temp/features_blue_merge_BMI_addfea_handlemissing.csv')


count_missing_values: data/temp/features_red_merge_BMI_addfea_handlemissing.csv
columnname missing_row_count missing_MaskID_count
MaskID 0 0
Visit 0 0
glycemia 0 0
bp 0 0
sbp 206780 8960
dbp 206802 8960
hr 207019 8963
hba1c 252121 10236
TC 319710 10240
trig 319710 10240
vldl 319720 10240
ldl 319720 10240
hdl 319718 10240
fpg 309754 10240
alt 301734 10201
cpk 330554 10214
potassium 325866 10240
screat 279082 10240
gfr 279084 10240
ualb 353124 10240
ucreat 353123 10240
uacr 354555 10241
edu_baseline 240 7
yrsdiab_baseline 3301 92
yrstens_baseline 93066 2525
cigarett_baseline 0 0
wt_kg_baseline 100 2
ht_cm_baseline 110 4
wt_kg_visit 329474 10239
ht_cm_visit 329731 10239
oral_gmed 0 0
medadd 0 0
medchg_intbp 0 0
medchg_stdbp 0 0
bp_med 0 0
BMI 329900 10239
female 0 0
baseline_age 0 0
cvd_hx_baseline 0 0
raceclass 0 0
type_po 0 0

MaskID_to_remove = {100001, 100002, 100003, 100004, 100005, 100006, 100007, 100008, 100009, 100010, 100011, 100012, 100013, 100014, 100015, 100016, 100017, 10001

Total 10251 patients:
missing sbp: 8960 people
missing edu_baseline: 7 people
missing BMI: 10239 people (have partial visit record)

sbp and other numeric values didnot fill in with average of adjacent values.

# STOP

## Calculate CVD_risk

In [22]:
def add_CVDrisk(fn):

    print('\nadd_CVDrisk:', fn)
    df = pd.read_csv(fn)
    CVD = []
    for i in tqdm(range(df.shape[0])):
        row = df.iloc[i]

        # collect the features
        
        # following features have no missinng rows
        race = 'Black' if row['raceclass']=='Black' else 'White/Other'
        gender = 'Female' if row['female']==1 else 'Male'
        age = row['baseline_age']
        smoke = 1 if row['cigarett_baseline'] == 1 else 0 # 1 means yes, 2 means no

        # patients with missing values in the following features are already removed
        TC = 1 if row['TC']==-1 else row['TC'] # 96 rows have value -1, which means missing, take 1 when missing such that log(1) = 0
        HDL = 1 if row['hdl']==-1 else row['hdl'] # 96 rows have value -1, which means missing
        SBP = 1 if row['sbp']==-1 else row['sbp'] # 3 rows have value -1, which means missing

        if race=='Black' and gender=='Female':
            q = np.exp( 17.114*np.log(age) + 0.940*np.log(TC) - 18.920*np.log(HDL) \
                        + 4.475*np.log(age)*np.log(HDL) \
                        + 29.291*np.log(SBP) - 6.432*np.log(age)*np.log(SBP) \
                        + 0.691*smoke +0.874*(1) -86.61)
            p_ascvd_10year = 1-np.power(0.9533, q)

        elif race=='White/Other' and gender=='Female':
            q = np.exp( -29.799*np.log(age) + 4.884*np.log(age)*np.log(age) +13.54*np.log(TC) \
               - 3.114*np.log(age)*np.log(TC) - 13.578*np.log(HDL) + 3.149*np.log(age)*np.log(HDL) \
               + 2.019*np.log(SBP) + 7.574*smoke - 1.665*np.log(age)*smoke +0.661*(1) + 29.18)

            p_ascvd_10year = 1-np.power(0.9665, q)
        
        elif race=='White/Other' and gender=='Male':
            q = np.exp( 12.344*np.log(age) +11.853*np.log(TC) \
               -2.664*np.log(age)*np.log(TC) -7.990*np.log(HDL) +1.769*np.log(age)*np.log(HDL) \
               +1.797*np.log(SBP)  \
               +7.837*smoke -1.795*np.log(age)*smoke +0.658*(1) -61.18)
            
            p_ascvd_10year = 1-np.power(0.9144, q)
        
        elif race=='Black' and gender=='Male':
            q = np.exp(2.469*np.log(age) +0.302*np.log(TC) -0.307*np.log(HDL)  \
               +1.916*np.log(SBP)  \
               +0.549*smoke +0.645*(1) -19.54)
            p_ascvd_10year = 1-np.power(0.8954, q)
        
        else:
            print('unrecognized race and gender:', race, gender)
            exit()
        
        CVD.append(p_ascvd_10year)
   
    df['CVDRisk'] = CVD
    print(df.info())

    df.to_csv(fn.split('.')[0]+'_addCVDrisk.csv', index=False)

add_CVDrisk('data/temp/features_red_merge_BMI_addfea_handlemissing.csv')
# add_CVDrisk('data/temp/features_blue_merge_BMI_addfea_handlemissing.csv')


add_CVDrisk: data/temp/features_red_merge_BMI_addfea_handlemissing.csv


100%|██████████| 388771/388771 [01:04<00:00, 5997.29it/s]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 388771 entries, 0 to 388770
Data columns (total 42 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   MaskID             388771 non-null  int64  
 1   Visit              388771 non-null  object 
 2   glycemia           375659 non-null  float64
 3   bp                 123919 non-null  float64
 4   sbp                388771 non-null  float64
 5   dbp                388771 non-null  float64
 6   hr                 388771 non-null  float64
 7   hba1c              388771 non-null  float64
 8   TC                 388771 non-null  float64
 9   trig               388771 non-null  float64
 10  vldl               388771 non-null  float64
 11  ldl                388771 non-null  float64
 12  hdl                388771 non-null  float64
 13  fpg                388771 non-null  float64
 14  alt                388771 non-null  float64
 15  cpk                388771 non-null  float64
 16  po

## Add BPClass.v2

 Now we fixed the data in `ACCORD_BPClass_v2_merged_Contextual.csv` such that the average values will be corrected by the missing values represented by "-1"

In [30]:
df = pd.read_csv('data/temp/features_red_merge_BMI_addfea_handlemissing_addCVDrisk.csv')
print(df.shape)

# build the dict of each column in df using MaskID and Visit as key
columns_to_replace = df.columns
columns_to_replace = columns_to_replace.drop('MaskID')
columns_to_replace = columns_to_replace.drop('Visit')
# convert to list
columns_to_replace = columns_to_replace.tolist()
print('len(columns_to_replace) =', len(columns_to_replace))
print('columns_to_replace =', columns_to_replace)

replace_data = {}
# loop through each row in df
for i in range(df.shape[0]):
    row = df.iloc[i]
    # get the key
    key = (row['MaskID'], row['Visit'])
    # loop through each column in df
    for col in columns_to_replace:
        # get the value of the cell
        value = row[col]

        # add the value to the dict
        if key not in replace_data:
            replace_data[key] = {}
        replace_data[key][col] = value

print('len(replace_data) =', len(replace_data))

(388771, 42)
len(columns_to_replace) = 40
columns_to_replace = ['glycemia', 'bp', 'sbp', 'dbp', 'hr', 'hba1c', 'TC', 'trig', 'vldl', 'ldl', 'hdl', 'fpg', 'alt', 'cpk', 'potassium', 'screat', 'gfr', 'ualb', 'ucreat', 'uacr', 'edu_baseline', 'yrsdiab_baseline', 'yrstens_baseline', 'cigarett_baseline', 'wt_kg_baseline', 'ht_cm_baseline', 'wt_kg_visit', 'ht_cm_visit', 'oral_gmed', 'medadd', 'medchg_intbp', 'medchg_stdbp', 'bp_med', 'BMI', 'female', 'baseline_age', 'cvd_hx_baseline', 'raceclass', 'type_po', 'CVDRisk']
len(replace_data) = 388771


In [31]:
import tqdm

df_target = pd.read_csv('../data/ACCORD_BPClass_v2_merged_Contextual.csv')
print(df_target.shape)

for col in df_target.columns:
    print(col, df_target[col].value_counts().get(-1, 0))

# loop through each row in df_target
for i in tqdm(range(df_target.shape[0])):
    row = df_target.iloc[i]
    # get the key
    key = (row['MaskID'], row['Visit'])
    # loop through each column in df_target
    for col in columns_to_replace:
        # get the value of the cell
        value = row[col]

        # replace the value in df_target
        if key in replace_data:
            df_target.loc[i, col] = replace_data[key][col]
        else:
            print('key not found:', key)
            exit()

# count the number of -1 in each column of df_target
for col in df_target.columns:
    print(col, df_target[col].value_counts().get(-1, 0))


(176880, 77)
MaskID 0
Visit 70
glycemia 0
bp 0
sbp 0
dbp 0
hr 0
hba1c 0
TC 0
trig 0
vldl 0
ldl 0
hdl 0
fpg 0
alt 0
cpk 121
potassium 0
screat 0
gfr 0
ualb 142
ucreat 142
uacr 541
edu_baseline 0
yrsdiab_baseline 1708
yrstens_baseline 35151
cigarett_baseline 0
wt_kg_baseline 0
ht_cm_baseline 0
wt_kg_visit 0
ht_cm_visit 0
oral_gmed 1
medadd 0
medchg_intbp 0
medchg_stdbp 0
bp_med 1
BMI 0
female 0
baseline_age 0
cvd_hx_baseline 0
raceclass 12169
type_po 0
CVDRisk 0
BPClass 1
sbp_discrete 0
hba1c_discrete 0
BMI_discrete 0
hdl_discrete 0
TC_discrete 0
sbp_feedback 0
hba1c_feedback 0
CVDRisk_feedback 0
bpclass_none 0
Diur 0
ACE 0
Beta-blocker 0
CCB 0
ARB 0
Alpha-Beta-blocker 0
Alpha-blocker 0
Sympath 0
Vasod 0
baseline_BMI 0
race_whiteother 0
race_black 0
CVDRisk_feedback_binary 0
BMI_feedback 0
TC_feedback 0
hdl_feedback 0
sbp_feedback_discrete 0
hba1c_feedback_discrete 0
BMI_feedback_discrete 0
hdl_feedback_discrete 0
TC_feedback_discrete 0
sbp_discrete_merged 0
action_code 0
state_code 0
ba

In [35]:
# save the df_target to csv
df_target.to_csv('../data/ACCORD_BPClass_v2_merged_Contextual_-1NotAvg.csv', index=False)

In [37]:
# get rows with Visit=-1 in df_target
df_target_error = df_target[df_target['BMI']==-1]
print(df_target_error.shape)

print(df_target['Visit'].value_counts().get(-1, 0))

# print(df_target_error.head())

(151801, 77)
70


# STOP---------------------

# ACCORD_BPMed.csv

## Analyze Medicine Combo frequency

In [29]:
# read in the table with added features / read in the f15_bptrialmedicationslog_medlist.csv
# count the frequency of each value in the bp_med column, notice the value is a string

def count_med_freq(fn, col):
    print('\ncount_med_freq:', fn, col)
    df = pd.read_csv(fn)
    print(df.shape)
    # print(df.info())

    # count the frequency of each value in the bp_med column
    med_freq_dict = {}
    for i in tqdm(range(df.shape[0])):
        row = df.iloc[i]
        med = row[col]
        # print('med =', med)

        # check if there is duplicate in med
        # med_list = med.split('+') # convert med to a list
        # med_set = set(med_list)
        # if (len(med_set) != len(med_list)):
        #     print('error: duplicate in med_list =', med_list)
        #     print('med_set =', med_set)
        #     exit()
        
        # count the frequency using str as key
        if med in med_freq_dict:
            med_freq_dict[med] += 1
        else:
            med_freq_dict[med] = 1

    # sort the med_freq_dict by decreasing value
    med_freq_dict = {k: v for k, v in sorted(med_freq_dict.items(), key=lambda item: item[1], reverse=True)}
    print('len(med_freq_dict) =', len(med_freq_dict))

    # convert the med_freq_dict to a pandas dataframe, use key as a column, and value as another column
    med_key = list(med_freq_dict.keys())
    med_val = list(med_freq_dict.values())
    med_length = [len(med.split('+')) for med in med_key]
    med_freq_df = pd.DataFrame({col: med_key, 'freq': med_val, 'length': med_length})
    med_freq_df.to_csv('data/temp/' + col + '_combo_freq.csv', index=False)
    print(med_freq_df.head(10))

# count_med_freq('data/features_red/f15_bptrialmedicationslog_medlist.csv', 'bp_med')
# count_med_freq('data/features_red/f10_glycemiamedicationslog_medlist.csv', 'oral_gmed')
count_med_freq('data/ACCORD.csv', 'bp_med')
count_med_freq('data/ACCORD.csv', 'oral_gmed')


count_med_freq: data/ACCORD.csv bp_med
(388523, 42)
len(med_freq_dict) = 4986
                                 bp_med    freq  length
0                                  none  287496       1
1                            lisinopril    4538       1
2                       hctz+lisinopril    4018       2
3                              ramipril    3449       1
4            hctz+lisinopril+metoprolol    2909       3
5             chlorthalidone+lisinopril    2078       2
6               chlorthalidone+ramipril    1978       2
7                        hctz+valsartan    1903       2
8  chlorthalidone+lisinopril+metoprolol    1497       3
9                 lisinopril+metoprolol    1488       2

count_med_freq: data/ACCORD.csv oral_gmed
(388523, 42)
len(med_freq_dict) = 439
                                         oral_gmed   freq  length
0                                             none  88205       1
1              glimepiride+metformin+rosiglitazone  65290       3
2                         

## Check for patients who have used "other xxx"

In [24]:
# get the patient MaskID set who had ever used the "other xxx" in their BP_med column

global BP_other_med_names    
BP_other_med_names = set()

def check_other(med_list):
    has_other_med = False
    for med_visit in med_list:
        med_names_list = med_visit.split('+')
        for med_name in med_names_list: 
            if med_name[:5]=='other':
                has_other_med = True
                BP_other_med_names.add(med_name)
                return has_other_med

    return has_other_med

def check_other_med(fn):
    print('\ncheck_med_freq:', fn)
    df = pd.read_csv(fn)
    print(df.shape)    

    BP_other_patients = set()
    # loop through each patient
    for maskid in tqdm(range(100001, 110252)):
        # print('maskid =', maskid)
        df_patient = df[df['MaskID'] == maskid]
        patient_bpmed = df_patient['bp_med'].tolist()
        
        has_other_bpmed = check_other(patient_bpmed)

        if has_other_bpmed:
            BP_other_patients.add(maskid)
            # BP_other_med_names.add(med_name)
        else:
            pass


    
    print('len(BP_other_patients) =', len(BP_other_patients))
    # check how many rows in df are in BP_other_patients
    df_BP_other_patients = df[df['MaskID'].isin(BP_other_patients)]
    print('df_BP_other_patients.shape =', df_BP_other_patients.shape)

    print('len(BP_other_med_names) =', len(BP_other_med_names))
    print('BP_other_med_names =', BP_other_med_names)

check_other_med('data/ACCORD.csv')
# check_other_med('data/temp/features_red_merge_BMI_addfea_handlemissing_removepatients_addCVDrisk.csv')


check_med_freq: data/ACCORD.csv
(388523, 42)


100%|██████████| 10251/10251 [00:06<00:00, 1588.42it/s]


len(BP_other_patients) = 2799
df_BP_other_patients.shape = (112736, 42)
len(BP_other_med_names) = 7
BP_other_med_names = {'other anti-hypertensive', 'other beta blocker', 'other ace inhibitor', 'other diuretic', 'other ccb', 'other alpha blocker', 'other a ii rb'}


In [ ]:
We would remove 2799 patients (112736/388523 rows) if we remove patients who had any usage of 'other xx' BP_med.

## Check how many rows have 'other xxx' in bp_med

Also, check how many rows have more than 5 bp medications, and crresponding to how many patients?

In [30]:
def check_other(med_list):
    has_other_med = False
    for med_visit in med_list:
        med_names_list = med_visit.split('+')
        for med_name in med_names_list: 
            if med_name[:5]=='other':
                has_other_med = True
                BP_other_med_names.add(med_name)
                return has_other_med

    return has_other_med

def check_other_med_rows(fn):
    print('\ncheck_med_freq:', fn)
    df = pd.read_csv(fn)
    print(df.shape)    

    BP_other_rows = set()
    morethan_5bp_rows = set()
    # loop through each row
    for i in tqdm(range(df.shape[0])):
        row = df.iloc[i]
        bpmed_names_list = row['bp_med'].split('+')
        if len(bpmed_names_list) > 5:
            morethan_5bp_rows.add(i)

        for bpmed_name in bpmed_names_list:
            if bpmed_name[:5]=='other':
                BP_other_rows.add(i)
                break
    
    print('len(BP_other_rows) =', len(BP_other_rows))

    # keep only rows that are in BP_other_rows
    df_other_rows = df[df.index.isin(BP_other_rows)]
    print('df_other_rows.shape =', df_other_rows.shape)

    # check how many different patients in df_other_rows
    BP_other_patients = set(df_other_rows['MaskID'].tolist())
    print('len(BP_other_patients) =', len(BP_other_patients))



    # check how many rows in df are in more than 5 bpmed rows
    df_morethan_5bp_rows = df[df.index.isin(morethan_5bp_rows)]
    print('df_morethan_5bp_rows.shape =', df_morethan_5bp_rows.shape)

    # check how many different patients in df_morethan_5bp_rows
    morethan_5bp_patients = set(df_morethan_5bp_rows['MaskID'].tolist())
    print('len(morethan_5bp_patients) =', len(morethan_5bp_patients))

    # check how many rows in df with MaskID in morethan_5bp_patients
    df_morethan_5bp_patients = df[df['MaskID'].isin(morethan_5bp_patients)]
    print('df_morethan_5bp_patients.shape =', df_morethan_5bp_patients.shape)

check_other_med_rows('data/ACCORD.csv')


check_med_freq: data/ACCORD.csv
(388523, 42)


100%|██████████| 388523/388523 [00:44<00:00, 8747.63it/s]


len(BP_other_rows) = 14807
df_other_rows.shape = (14807, 42)
len(BP_other_patients) = 2799
df_morethan_5bp_rows.shape = (4814, 42)
len(morethan_5bp_patients) = 884
df_morethan_5bp_patients.shape = (38044, 42)


## Check the frequency of 17 BP medications and 13 BG medications

In [26]:
def check_med_freq(fn, BGMED_freq_dict, BPMED_freq_dict):
    print('\ncheck_med_freq:', fn)
    df = pd.read_csv(fn)
    print(df.shape)
   
    # loop through each row/visit
    for i in tqdm(range(df.shape[0])):
        row = df.iloc[i]
        BPMED = row['bp_med']
        BGMED = row['oral_gmed']
        # print('BGMED =', BGMED)
        # print('BPMED =', BPMED)

        BGMED_list = BGMED.split('+') # convert BGMED to a list
        for med in BGMED_list:
            if med in BGMED_freq_dict:
                BGMED_freq_dict[med] += 1
            else:
                # continue
                BGMED_freq_dict[med] = 1

        BPMED_list = BPMED.split('+') # convert BPMED to a list
        for med in BPMED_list:
            if med in BPMED_freq_dict:
                BPMED_freq_dict[med] += 1
            else:
                # continue
                BPMED_freq_dict[med] = 1

    # sort the med_freq_dict by decreasing value
    BGMED_freq_dict = {k: v for k, v in sorted(BGMED_freq_dict.items(), key=lambda item: item[1], reverse=True)}
    BPMED_freq_dict = {k: v for k, v in sorted(BPMED_freq_dict.items(), key=lambda item: item[1], reverse=True)}

    # save the BGMED_freq_dict and BPMED_freq_dict to csv
    BGMED_key = list(BGMED_freq_dict.keys())
    BGMED_val = list(BGMED_freq_dict.values())
    BGMED_freq_df = pd.DataFrame({'BGMED': BGMED_key, 'freq': BGMED_val})
    BGMED_freq_df.to_csv('data/temp/BGMED_freq.csv', index=False)

    BPMED_key = list(BPMED_freq_dict.keys())
    BPMED_val = list(BPMED_freq_dict.values())
    BPMED_freq_df = pd.DataFrame({'BPMED': BPMED_key, 'freq': BPMED_val})
    BPMED_freq_df.to_csv('data/temp/BPMED_freq.csv', index=False)
    

# 13 BG meds
BGMED_list = ['gliclazide','glimepiride','glipizide','glyburide','metformin',
              'exanatide','repaglinide','nateglinide','rosiglitazone','pioglitazone',
              'sitagliptin','acarbose','pramlintide']

# 17 BP meds
BPMED_list = ['chlorthalidone','hctz', 'triamterene', 'furosemide', 'candesartan',
              'valsartan', 'benazepril', 'lisinopril','ramipril', 'felodipine',
              'amlodipine','diltiazem', 'terazosin','metoprolol', 'carvedilol',
              'reserpine','hydralazine']

BGMED_freq_dict = {k:v for k,v in zip(BGMED_list, [0]*len(BGMED_list))}
BPMED_freq_dict = {k:v for k,v in zip(BPMED_list, [0]*len(BPMED_list))}

check_med_freq('data/ACCORD.csv', BGMED_freq_dict, BPMED_freq_dict)
# check_med_freq('data/temp/features_blue_merge_BMI_addfea_handlemissing.csv', BGMED_freq_dict, BPMED_freq_dict)


check_med_freq: data/ACCORD.csv
(388523, 42)


100%|██████████| 388523/388523 [00:45<00:00, 8477.14it/s]


## Remove any patients who has ever used "other anti-hypertensive"

In [33]:
def remove_anti_hypertensive_meds(fn):
    df = pd.read_csv(fn)
    print(df.shape)

    antihypertensive_rows = []
    antihypertensive_patients = set()
    # loop through each row
    for i in tqdm(range(df.shape[0])):
        row = df.iloc[i]
        BPMED = row['bp_med']
        BPMED_list = BPMED.split('+')
        BPMED_set = set(BPMED_list)

        if 'other anti-hypertensive' in BPMED_set:
            antihypertensive_rows.append(i)
            antihypertensive_patients.add(row['MaskID'])
    
    print('# of visit rows containing "other anti-hypertensive"=', len(antihypertensive_rows))
    print('# of patients with "other anti-hypertensive"=', len(antihypertensive_patients))

    # remove rows whose MaskID is in antihypertensive_patients
    df_other_rows = df[~df['MaskID'].isin(antihypertensive_patients)]

    # save df_other_rows to csv
    df_other_rows.to_csv('data/temp/ACCORD_removeantihypertensive.csv', index=False)
    print('df_other_rows.shape =', df_other_rows.shape)
    print("# of visit rows removed =", df.shape[0] - df_other_rows.shape[0])

remove_anti_hypertensive_meds('data/ACCORD.csv')

(388523, 42)


100%|██████████| 388523/388523 [00:47<00:00, 8194.57it/s]


# of visit rows containing "other anti-hypertensive"= 1246
# of patients with "other anti-hypertensive"= 314
df_other_rows.shape = (375317, 42)
# of visit rows removed = 13206


## Generate ACCORD.BPMed.csv

1. reorder each medvisit by its medicine-frequency from high to low, throw away any >5 BP medicine
2. replace any "other xxx" with the most-frequent medicine under the same category, if highest-freq results duplicates, then take the 2nd freq med, etc.
3. keep the original bp_med column, name the new column 'bp_med_most5'

In [46]:
def process_bpmed_by_freq(fn, col_name, MED_freq_dict, other_bp_med_dict):
    """
    Sort the MED_list by decreasing freq, 
    truncate to top 5 meds with highest freq, 
    sort again by alphabetical order
    """
    df = pd.read_csv(fn)
    print(df.shape)

    duplicate_ct = 0
    med_new_list = []
    # loop through each row/visit
    for i in tqdm(range(df.shape[0])):
        row = df.iloc[i]
        MED = row['bp_med']
        MED_list = MED.split('+')
        
        # if len(MED_list)>5, sort the MED_list by decreasing freq
        if len(MED_list) > 5:
            MED_list.sort(key=lambda x: MED_freq_dict[x], reverse=True)
            MED_list = MED_list[:5] # truncate to top 5 meds with highest freq
        
        # replace any 'other xxx' with corresponding highest freq med under same category
        for i in range(len(MED_list)):
            if MED_list[i] in other_bp_med_dict:
                replace = other_bp_med_dict[MED_list[i]][0] # replace with the highest freq med
                med_set = set(MED_list)
                if replace in med_set:
                    replace = other_bp_med_dict[MED_list[i]][1] # replace with the 2nd highest freq med
                if replace in med_set:
                    replace = other_bp_med_dict[MED_list[i]][2] # replace with the 3rd highest freq med
                if replace in med_set:
                    replace = other_bp_med_dict[MED_list[i]][3] # replace with the 4th highest freq med
                MED_list[i] = replace
        
        # check if there are any duplicates in MED_list
        MED_set = set(MED_list)
        # MED_list = list(MED_set)
        if len(MED_set) < len(MED_list):

            duplicate_ct += 1
            print('MED=', MED)
            print('MED_list=', MED_list)
            print('MED_set=', MED_set)
            raise Exception('MED_list has duplicates')
        
        # remove '' item in MED_list
        MED_list = [x for x in MED_list if x != '']

        # sort again by alphabetical order based on the truncated and replaced MED_list
        MED_list.sort()
        MED_new = '+'.join(MED_list)
        med_new_list.append(MED_new)
    
    df['{}_most5'.format(col_name)] = med_new_list

    # save df to csv
    df.to_csv('data/temp/ACCORD_removeantihypertensive_most5.csv', index=False)
    print('df.shape =', df.shape)
    print("# of rows has duplicates =", duplicate_ct)



# read in the 'data/temp/BPMED_freq.csv' into a dictionary
BPMED_freq_dict = {}
df = pd.read_csv('data/temp/BPMED_freq.csv')
for i in range(df.shape[0]):
    row = df.iloc[i]
    BPMED_freq_dict[row['BPMED']] = row['freq']
print(BPMED_freq_dict)

other_bp_med_dict = {'other ccb': ['amlodipine','diltiazem','felodipine'], 
                     'other diuretic':['hctz','chlorthalidone','furosemide','triamterene'], 
                     'other beta blocker':['metoprolol', ''], 
                     'other ace inhibitor':['lisinopril', 'benazepril', 'ramipril'], 
                     'other a ii rb': ['valsartan','candesartan'], 
                     'other alpha blocker': ['terazosin', '']}

process_bpmed_by_freq('data/temp/ACCORD_removeantihypertensive.csv', 'bp_med', BPMED_freq_dict, other_bp_med_dict)

{'none': 287496, 'metoprolol': 40506, 'hctz': 34493, 'lisinopril': 34222, 'chlorthalidone': 31587, 'valsartan': 19483, 'benazepril': 16473, 'furosemide': 15870, 'candesartan': 15025, 'ramipril': 14858, 'amlodipine': 13203, 'diltiazem': 13198, 'carvedilol': 8049, 'felodipine': 7603, 'terazosin': 7091, 'reserpine': 5725, 'other ccb': 5149, 'other diuretic': 4770, 'triamterene': 3887, 'hydralazine': 3229, 'other beta blocker': 3021, 'other ace inhibitor': 2240, 'other a ii rb': 1542, 'other anti-hypertensive': 1246, 'other alpha blocker': 1108}
(375317, 42)


100%|██████████| 375317/375317 [00:46<00:00, 8152.20it/s]


df.shape = (375317, 43)
# of rows has duplicates = 0


## Copy file 'data/temp/ACCORD_removeantihypertensive_most5.csv' to 'data/ACCORD_BPMed.csv'

## NOT USED -  Remove any patients who has ever used other medications

Also checked how many patients had at least 1 visit with more than 5 medications.

In [107]:
def check_patient_med_5(patient_med_list):

    has_morethan_5med = False
    for med_visit in patient_med_list:
        med_list = med_visit.split('+')
        if len(med_list) > 5: # as long as one visit has more than 5 meds, return True
            return True
        else:
            continue
            
    return has_morethan_5med


def check_patient_med(patient_med_list, MED_set):

    has_other_med = False
    for med_visit in patient_med_list:
        med_list = med_visit.split('+')
        for med in med_list:
            if med[:5]=='other': # consider 'other' in the med_set
                continue
            elif med not in MED_set:
                return True
            else:
                continue

    return has_other_med

def check_other_med(fn, BPMED_set, BGMED_set):

    print('\ncheck_other_med:', fn)
    df = pd.read_csv(fn)
    print(df.shape)

    BP_to_remove = set()
    BG_to_remove = set()

    BP_to_remove5 = set()
    BG_to_remove5 = set()

    # loop through each patient
    for maskid in tqdm(range(100001, 110252)):
        # print('maskid =', maskid)
        df_patient = df[df['MaskID'] == maskid]
        patient_bpmed = df_patient['bp_med'].tolist()
        patient_bgmed = df_patient['oral_gmed'].tolist()

        has_other_bp = check_patient_med(patient_bpmed, BPMED_set)
        has_other_bg = check_patient_med(patient_bgmed, BGMED_set)

        has_5_bp = check_patient_med_5(patient_bpmed)
        has_5_bg = check_patient_med_5(patient_bgmed)

        if has_other_bp == True:
            BP_to_remove.add(maskid)
        if has_other_bg == True:
            BG_to_remove.add(maskid)
        
        if has_5_bp == True:
            BP_to_remove5.add(maskid)
        if has_5_bg == True:
            BG_to_remove5.add(maskid)
    
    # count how many rows with MaskID in BP_to_remove and BG_to_remove
    print('\nlen(BP_to_remove) =', len(BP_to_remove))
    df_bp_to_remove = df[df['MaskID'].isin(BP_to_remove)]
    print('df_bp_to_remove.shape =', df_bp_to_remove.shape)
        
    print('\nlen(BG_to_remove) =', len(BG_to_remove))
    df_bg_to_remove = df[df['MaskID'].isin(BG_to_remove)]
    print('df_bg_to_remove.shape =', df_bg_to_remove.shape)

    print('\nlen(BP_to_remove5) =', len(BP_to_remove5))
    df_bp_to_remove5 = df[df['MaskID'].isin(BP_to_remove5)]
    print('df_bp_to_remove5.shape =', df_bp_to_remove5.shape)

    print('\nlen(BG_to_remove5) =', len(BG_to_remove5))
    df_bg_to_remove5 = df[df['MaskID'].isin(BG_to_remove5)]
    print('df_bg_to_remove5.shape =', df_bg_to_remove5.shape)

BGMED_set = set(BGMED_list)
BGMED_set.add('none')
BPMED_set = set(BPMED_list)
BPMED_set.add('none') # need to consider 'none' case

check_other_med('data/temp/features_red_merge_BMI_addfea_handlemissing.csv', BGMED_set, BPMED_set)


check_other_med: data/temp/features_red_merge_BMI_addfea_handlemissing.csv
(388771, 41)


100%|██████████| 10251/10251 [00:07<00:00, 1453.21it/s]



len(BP_to_remove) = 4660
df_bp_to_remove.shape = (189808, 41)

len(BG_to_remove) = 10055
df_bg_to_remove.shape = (384139, 41)

len(BP_to_remove5) = 884
df_bp_to_remove5.shape = (38044, 41)

len(BG_to_remove5) = 45
df_bg_to_remove5.shape = (2332, 41)


# ACCORD_BGMed.csv

## check how many rows and patients related to removing less-frequent BG_Med

In [49]:
def process_bg_med(fn, bg_med_to_remove):

    print('\nprocess_bg_med:', fn)
    df = pd.read_csv(fn)
    print(df.shape)

    row_id_to_remove = []
    MaskID_to_remove = set()
    row_morethan5 = []
    MaskID_moretthan5 = set()
    # loop through each row
    for i in tqdm(range(df.shape[0])):
        row = df.iloc[i]
        patient_bgmed = row['oral_gmed'].split('+')        
        if len(patient_bgmed) > 5:
            row_morethan5.append(i)
            MaskID_moretthan5.add(row['MaskID'])
            
        for med in patient_bgmed:
            if med in bg_med_to_remove:
                row_id_to_remove.append(i)
                MaskID_to_remove.add(row['MaskID'])
                break
    
    print('# of rows containing less freq bg_med =', len(row_id_to_remove))
    print('# of associated patients =', len(MaskID_to_remove))

    # count how many rows with MaskID in BG_to_remove
    df_to_remove = df[df['MaskID'].isin(MaskID_to_remove)]
    print('# of total rows removed if remove assocaiated patients =', df_to_remove.shape)

    # remove patients with MaskID in BG_to_remove
    df_keep = df[~df['MaskID'].isin(MaskID_to_remove)]
    print('after removing associated patients, df_keep.shape =', df_keep.shape)


    # more than 5 bg_med
    print('\n# of rows containing more than 5 bg_med =', len(row_morethan5))
    print('# of associated patients =', len(MaskID_moretthan5))
    df_morethan5 = df[df['MaskID'].isin(MaskID_moretthan5)]
    print('# of total rows removed if remove assocaiated patients =', df_morethan5.shape)

    # remove patients with MaskID in MaskID_moretthan5
    df_keep = df_keep[~df_keep['MaskID'].isin(MaskID_moretthan5)]
    print('after removing associated morethan5 patients, df_keep.shape =', df_keep.shape)

    df_keep.to_csv('data/temp/ACCORD_BPMed_BGMed.csv', index=False)

bg_med_to_remove = {'nateglinide','gliclazide', 'pramlintide', 'sulfonylurea'}
process_bg_med('data/ACCORD_BPMed.csv', bg_med_to_remove)


process_bg_med: data/ACCORD_BPMed.csv
(375317, 43)


100%|██████████| 375317/375317 [00:47<00:00, 7852.63it/s]


# of rows containing less freq bg_med = 2030
# of associated patients = 499
# of total rows removed if remove assocaiated patients = (21771, 43)
after removing associated patients, df_keep.shape = (353546, 43)

# of rows containing more than 5 bg_med = 101
# of associated patients = 43
# of total rows removed if remove assocaiated patients = (2239, 43)
after removing associated morethan5 patients, df_keep.shape = (351481, 43)


Copy file 'data/temp/ACCORD_BPMed_BGMed.csv' to 'data/ACCORD_BGMed.csv'

# ACCORD_BPBGMed.csv

* select only BP active patients
* 17 BP med, 10 BG med

In [52]:
def process_bpbg_med(fn, bp_active_set):
    df = pd.read_csv(fn)
    print(df.shape)

    # select only rows with MaskID in bp_active_set
    df = df[df['MaskID'].isin(bp_active_set)]
    print('after selecting only rows with MaskID in bp_active_set, df.shape =', df.shape)

    # save to csv
    df.to_csv('data/ACCORD_BPBGMed.csv', index=False)


# load bp_active_set from the csv file, each line is a MaskID
with open('data/temp/candidates_bp.csv', 'r') as f:
    bp_active_set = set()
    for line in f:
        bp_active_set.add(int(line.strip()))
    print('len(bp_active_set) =', len(bp_active_set))

process_bpbg_med('data/ACCORD_BGMed.csv', bp_active_set)

len(bp_active_set) = 4733
(351481, 43)
after selecting only rows with MaskID in bp_active_set, df.shape = (167939, 43)


# ACCORD_BPClass.csv

## Convert bp_med to BPClass

In [3]:
def med_to_class(df, col_name, col_name_new, class_dict):

    # add new column 'BPClass'
    med_class = []

    # loop through each row
    for i in tqdm(range(df.shape[0])):
        row = df.iloc[i]
        med = row[col_name]
        if med != 'none':
            med_list = med.split('+')
            class_set = set()
            for med in med_list:
                if med in class_dict:
                    class_set.add(class_dict[med])
                else:
                    print('error: med not in class_dict:', med)
                    raise Exception('med not in class_dict')

            class_list = list(class_set)
            # if len(class_list) > 5:
            #     print('error: len(class_list) > 5')
            #     print('class_list =', class_list)
            #     exit()

            class_list.sort()
            med_class.append('+'.join(class_list))
        else:
            med_class.append('none')
    
    df[col_name_new] = med_class
    return df
    
def process_BPclass(fn, class_dict):
    df = pd.read_csv(fn)
    print(df.shape)

    df = med_to_class(df, 'bp_med', 'BPClass', class_dict)

    df.to_csv('data/temp/ACCORD_BPMed2Class.csv', index=False)

# load bp_class_dict from the csv file, keys are bp_med column, values are class column
df = pd.read_csv('data/temp/bp_class_dict.csv')
bp_class_dict = {k:v for k,v in zip(df['bp_med'], df['class'])}
print('bp_class_dict =', bp_class_dict)

# notice here we use the file with anti-hypertensive removed from ACCORD.csv as a starting point
process_BPclass('data/temp/ACCORD_removeantihypertensive.csv', bp_class_dict)

bp_class_dict = {'chlorthalidone': 'Diur', 'hctz': 'Diur', 'triamterene': 'Diur', 'furosemide': 'Diur', 'other diuretic': 'Diur', 'candesartan': 'ARB', 'valsartan': 'ARB', 'other a ii rb': 'ARB', 'benazepril': 'ACE', 'lisinopril': 'ACE', 'ramipril': 'ACE', 'other ace inhibitor': 'ACE', 'felodipine': 'CCB', 'amlodipine': 'CCB', 'diltiazem': 'CCB', 'other ccb': 'CCB', 'terazosin': 'Alpha-blocker', 'other alpha blocker': 'Alpha-blocker', 'metoprolol': 'Beta-blocker', 'other beta blocker': 'Beta-blocker', 'carvedilol': 'Alpha-Beta-blocker', 'reserpine': 'Sympath', 'hydralazine': 'Vasod'}
(375317, 42)


100%|██████████| 375317/375317 [00:42<00:00, 8934.60it/s]


## Delete non-active BP patients from dataset

In [10]:
def delete_nonactive_bp(fn, fn_new, bp_active_set):
    df = pd.read_csv(fn)
    print(df.shape)

    # select only rows with MaskID in bp_active_set
    df = df[df['MaskID'].isin(bp_active_set)]
    print('after selecting only rows with MaskID in bp_active_set, df.shape =', df.shape)

    # save to csv
    df.to_csv(fn_new, index=False)

# load bp_active_set from the csv file, each line is a MaskID
with open('data/temp/candidates_bp.csv', 'r') as f:
    bp_active_set = set()
    for line in f:
        bp_active_set.add(int(line.strip()))
    print('len(bp_active_set) =', len(bp_active_set))
    
delete_nonactive_bp('data/ACCORD_BPMed.csv', 'data/ACCORD_BPMed.csv', bp_active_set)
delete_nonactive_bp('data/temp/ACCORD_BPMed2Class.csv', 'data/temp/ACCORD_BPMed2Class_activeBP.csv', bp_active_set)

len(bp_active_set) = 4733
(178792, 43)
after selecting only rows with MaskID in bp_active_set, df.shape = (178792, 43)
(375317, 43)
after selecting only rows with MaskID in bp_active_set, df.shape = (178792, 43)


## Count frequency of each BPClass and BPClass combo

Delete patients who had more than 6 bp classes

In [12]:
df = pd.read_csv('data/temp/ACCORD_BPMed2Class_activeBP.csv')
print(df.shape)

bp_class = dict()
bp_class_combo = dict()
MaskID_morethan6 = set()
rows_morethan6 = []
# loop through each row
for i in tqdm(range(df.shape[0])):
    row = df.iloc[i]
    med = row['BPClass']
    if med not in bp_class_combo:
        bp_class_combo[med] = 1
    else:
        bp_class_combo[med] += 1

    class_list = med.split('+')
    for c in class_list:
        if c not in bp_class:
            bp_class[c] = 1
        else:
            bp_class[c] += 1

    if len(class_list) > 6:
        MaskID_morethan6.add(row['MaskID'])
        rows_morethan6.append(i)

print('len(bp_class) =', len(bp_class))
print('len(bp_class_combo) =', len(bp_class_combo))

print('len(MaskID_morethan6) =', len(MaskID_morethan6))

# count how many rows whose MaskID is in MaskID_morethan6
df_morethan6 = df[df['MaskID'].isin(MaskID_morethan6)]
print('len(rows_morethan6) =', len(rows_morethan6))
print('# of rows whose MaskID is in MaskID_morethan6 =', df_morethan6.shape[0])

df_keep = df[~df['MaskID'].isin(MaskID_morethan6)]
print('df_keep.shape =', df_keep.shape)
df_keep.to_csv('data/temp/ACCORD_BPMed2Class_activeBP_removemorethan6class.csv', index=False)

# sort by value
bp_class = {k: v for k, v in sorted(bp_class.items(), key=lambda item: item[1], reverse=True)}
bp_class_combo = {k: v for k, v in sorted(bp_class_combo.items(), key=lambda item: item[1], reverse=True)}

# save to csv
name = bp_class.keys()
freq = bp_class.values()
df = pd.DataFrame({'class':name, 'freq':freq})
df.to_csv('data/temp/bp_class.csv', index=False)

name = bp_class_combo.keys()
freq = bp_class_combo.values()
df = pd.DataFrame({'class_combo':name, 'freq':freq, 'length': [len(x.split('+')) for x in name]})
df.to_csv('data/temp/bp_class_combo.csv', index=False)

(178792, 43)


100%|██████████| 178792/178792 [00:20<00:00, 8578.19it/s]


len(bp_class) = 10
len(bp_class_combo) = 289
len(MaskID_morethan6) = 41
len(rows_morethan6) = 182
# of rows whose MaskID is in MaskID_morethan6 = 1761
df_keep.shape = (177031, 43)


Copy file '/data/temp/ACCORD_BPMed2Class_activeBP_removemorethan6class.csv' to 'data/ACCORD_BPClass.csv'

# ACCORD_BGClass.csv

## delete patients who had ever used any of the 4 bg med

bg_med_to_remove = {'nateglinide','gliclazide', 'pramlintide', 'sulfonylurea'}

In [15]:
bg_med_to_remove = {'nateglinide','gliclazide', 'pramlintide', 'sulfonylurea'}

df = pd.read_csv('data/temp/ACCORD_BPMed2Class.csv')
print(df.shape)

MaskID_to_remove = set()
for i in tqdm(range(df.shape[0])):
    row = df.iloc[i]
    med = row['oral_gmed']
    if med == 'none':
        continue

    class_list = med.split('+')
    for c in class_list:
        if c in bg_med_to_remove:
            MaskID_to_remove.add(row['MaskID'])
            break

print('len(MaskID_to_remove) =', len(MaskID_to_remove))

df_keep = df[~df['MaskID'].isin(MaskID_to_remove)]
print('df_keep.shape =', df_keep.shape)

df_keep.to_csv('data/temp/ACCORD_BPMed2Class_removebgmed.csv', index=False)

(375317, 43)


100%|██████████| 375317/375317 [00:43<00:00, 8696.30it/s]


len(MaskID_to_remove) = 499
df_keep.shape = (353546, 43)


## replace the bg med to BGClass

In [16]:
def process_BGclass(fn, class_dict):
    df = pd.read_csv(fn)
    print(df.shape)
    df = med_to_class(df, 'oral_gmed', 'BGClass', class_dict)
    df.to_csv('data/temp/ACCORD_BGMed2Class.csv', index=False)

# load the bg_class_dict from the csv file, keys are bg_med column, values are class column
df = pd.read_csv('data/temp/bg_class_dict.csv')
bg_class_dict = {k:v for k,v in zip(df['bg_med'], df['class'])}
print('bg_class_dict =', bg_class_dict)

process_BGclass('data/temp/ACCORD_BPMed2Class_removebgmed.csv', bg_class_dict)

bg_class_dict = {'gliclazide': 'Sulfon', 'glimepiride': 'Sulfon', 'glipizide': 'Sulfon', 'glyburide': 'Sulfon', 'metformin': 'Bingu', 'exanatide': 'Incr-ago', 'repaglinide': 'Meglit', 'nateglinide': 'Meglit', 'rosiglitazone': 'Thiaz', 'pioglitazone': 'Thiaz', 'sitagliptin': 'DPP-4', 'acarbose': 'Alpha-gluc', 'pramlintide': 'Amy-ana'}
(353546, 43)


100%|██████████| 353546/353546 [00:41<00:00, 8622.03it/s]


## Count frequency of each BPClass and BPClass combo

also remove patients who had ever used more than 5 BGclass

In [19]:
df = pd.read_csv('data/temp/ACCORD_BGMed2Class.csv')
print(df.shape)

bg_class = dict()
bg_class_combo = dict()
MaskID_morethan5 = set()
rows_morethan5 = []
# loop through each row
for i in tqdm(range(df.shape[0])):
    row = df.iloc[i]
    med = row['BGClass']
    if med not in bg_class_combo:
        bg_class_combo[med] = 1
    else:
        bg_class_combo[med] += 1

    class_list = med.split('+')
    for c in class_list:
        if c not in bg_class:
            bg_class[c] = 1
        else:
            bg_class[c] += 1

    if len(class_list) > 5:
        MaskID_morethan5.add(row['MaskID'])
        rows_morethan5.append(i)

print('len(bp_class) =', len(bp_class))
print('len(bp_class_combo) =', len(bp_class_combo))

print('len(MaskID_morethan5) =', len(MaskID_morethan5))

# count how many rows whose MaskID is in MaskID_morethan6
df_morethan5 = df[df['MaskID'].isin(MaskID_morethan5)]
print('len(rows_morethan5) =', len(rows_morethan5))
print('# of rows whose MaskID is in MaskID_morethan5 =', df_morethan5.shape[0])

df_keep = df[~df['MaskID'].isin(MaskID_morethan5)]
print('df_keep.shape =', df_keep.shape)
df_keep.to_csv('data/temp/ACCORD_BGMed2Class_removemorethan5class.csv', index=False)

# sort by value
bg_class = {k: v for k, v in sorted(bg_class.items(), key=lambda item: item[1], reverse=True)}
bg_class_combo = {k: v for k, v in sorted(bg_class_combo.items(), key=lambda item: item[1], reverse=True)}

# save to csv
name = bg_class.keys()
freq = bg_class.values()
df = pd.DataFrame({'class':name, 'freq':freq})
df.to_csv('data/temp/bg_class.csv', index=False)

name = bg_class_combo.keys()
freq = bg_class_combo.values()
df = pd.DataFrame({'class_combo':name, 'freq':freq, 'length': [len(x.split('+')) for x in name]})
df.to_csv('data/temp/bg_class_combo.csv', index=False)

(353546, 44)


100%|██████████| 353546/353546 [00:45<00:00, 7760.34it/s]


len(bp_class) = 10
len(bp_class_combo) = 289
len(MaskID_morethan5) = 28
len(rows_morethan5) = 78
# of rows whose MaskID is in MaskID_morethan5 = 1505
df_keep.shape = (352041, 44)


## remove patients who had ever used Incr-ago or DPP-4 BGClass

In [20]:
df = pd.read_csv('data/temp/ACCORD_BGMed2Class_removemorethan5class.csv')
print(df.shape)

bgclass_to_remove = {'Incr-ago', 'DPP-4'}

MaskID_to_remove = set()
for i in tqdm(range(df.shape[0])):
    row = df.iloc[i]
    med = row['BGClass']
    if med == 'none':
        continue

    class_list = med.split('+')
    for c in class_list:
        if c in bgclass_to_remove:
            MaskID_to_remove.add(row['MaskID'])
            break

print('len(MaskID_to_remove) =', len(MaskID_to_remove))

df_keep = df[~df['MaskID'].isin(MaskID_to_remove)]
print('df_keep.shape =', df_keep.shape)

df_keep.to_csv('data/temp/ACCORD_BGMed2Class_removemorethan5class_removebgclass.csv', index=False)


(352041, 44)


100%|██████████| 352041/352041 [00:41<00:00, 8448.54it/s]


len(MaskID_to_remove) = 1318
df_keep.shape = (290676, 44)


Copy 'data/temp/ACCORD_BGMed2Class_removemorethan5class_removebgclass.csv' to 'data/ACCORD_BGClass.csv'

# ACCORD_BPBGClass.csv

simply select active BP patients from 'data/ACCORD_BGClass.csv' obtained above

In [21]:
def process_bpbg_med(fn, bp_active_set):
    df = pd.read_csv(fn)
    print(df.shape)

    # select only rows with MaskID in bp_active_set
    df = df[df['MaskID'].isin(bp_active_set)]
    print('after selecting only rows with MaskID in bp_active_set, df.shape =', df.shape)

    # save to csv
    df.to_csv('data/ACCORD_BPBGClass.csv', index=False)


# load bp_active_set from the csv file, each line is a MaskID
with open('data/temp/candidates_bp.csv', 'r') as f:
    bp_active_set = set()
    for line in f:
        bp_active_set.add(int(line.strip()))
    print('len(bp_active_set) =', len(bp_active_set))

process_bpbg_med('data/ACCORD_BGClass.csv', bp_active_set)

len(bp_active_set) = 4733
(290676, 44)
after selecting only rows with MaskID in bp_active_set, df.shape = (139099, 44)


# v2.csv

1. discretize some numeric variables
2. add feedback
3. one-hot encode BPmed and BGmed

Discretize

In [23]:
def discretize_var(fn):

    print('fn =', fn)
    df = pd.read_csv(fn)
    print(df.shape)

    # discretize sbp, hba1c, BMI, hdl, TC
    sbp_discrete = []
    hba1c_discrete = []
    BMI_discrete = []
    hdl_discrete = []
    TC_discrete = []
    for i in tqdm(range(df.shape[0])):
        row = df.iloc[i]
        sbp = row['sbp']
        hba1c = row['hba1c']
        BMI = row['BMI']
        hdl = row['hdl']
        TC = row['TC']

        if sbp < 120:
            sbp_discrete.append(0)
        elif sbp < 140:
            sbp_discrete.append(1)
        elif sbp < 160:
            sbp_discrete.append(2)
        else:
            sbp_discrete.append(3)

        if hba1c < 6:
            hba1c_discrete.append(0)
        elif hba1c < 6.5:
            hba1c_discrete.append(1)
        elif hba1c < 7:
            hba1c_discrete.append(2)
        elif hba1c < 7.5:
            hba1c_discrete.append(3)
        elif hba1c < 8:
            hba1c_discrete.append(4)
        elif hba1c < 8.5:
            hba1c_discrete.append(5)
        elif hba1c < 9:
            hba1c_discrete.append(6)
        else:
            hba1c_discrete.append(7)            

        if BMI < 18.5:
            BMI_discrete.append(0)
        elif BMI < 25:
            BMI_discrete.append(1)
        elif BMI < 30:
            BMI_discrete.append(2)
        else:
            BMI_discrete.append(3)

        if hdl < 40:
            hdl_discrete.append(0)
        elif hdl < 50:
            hdl_discrete.append(1)
        elif hdl < 60:
            hdl_discrete.append(2)
        else:
            hdl_discrete.append(3)

        if TC < 160:
            TC_discrete.append(0)
        elif TC < 200:
            TC_discrete.append(1)
        elif TC < 240:
            TC_discrete.append(2)
        else:
            TC_discrete.append(3)
    
    df['sbp_discrete'] = sbp_discrete
    df['hba1c_discrete'] = hba1c_discrete
    df['BMI_discrete'] = BMI_discrete
    df['hdl_discrete'] = hdl_discrete
    df['TC_discrete'] = TC_discrete

    # save to csv
    fn_out = fn.replace('.csv', '_v2.csv')
    df.to_csv(fn_out, index=False)


discretize_var('data/ACCORD_BPMed.csv')
discretize_var('data/ACCORD_BGMed.csv')
discretize_var('data/ACCORD_BPBGMed.csv')
discretize_var('data/ACCORD_BPClass.csv')
discretize_var('data/ACCORD_BGClass.csv')  
discretize_var('data/ACCORD_BPBGClass.csv')


fn = data/ACCORD_BPMed.csv
(178792, 43)


100%|██████████| 178792/178792 [00:23<00:00, 7582.90it/s]



fn = data/ACCORD_BGMed.csv
(351481, 43)


100%|██████████| 351481/351481 [00:50<00:00, 6905.22it/s]



fn = data/ACCORD_BPBGMed.csv
(167939, 43)


...

100%|██████████| 177031/177031 [00:25<00:00, 7023.19it/s]



fn = data/ACCORD_BGClass.csv
(290676, 44)


100%|██████████| 290676/290676 [00:41<00:00, 7037.68it/s]



fn = data/ACCORD_BPBGClass.csv
(139099, 44)


100%|██████████| 139099/139099 [00:19<00:00, 7049.27it/s]


Add feedback

In [8]:
def get_feedback_patient(df_patient, col_name):

    # get df_patient[col_name] and convert it to list
    target_col = df_patient[col_name].tolist()
    res_col = target_col.copy()

    if len(target_col) == 1:
        return res_col
    
    for idx in range(len(target_col)-1):
        res_col[idx] = target_col[idx+1]
    res_col[-1] = target_col[-1]
    
    return res_col


def add_feedback(fn):
    print('fn =', fn)
    df = pd.read_csv(fn)
    print(df.shape)

    # add feedback
    sbp_fb = [] 
    hba1c_fb = []
    cvdrisk_fb = []
    bmi_fb = []
    TC_fb = []
    hdl_fb = []
    
    # loop through each patient
    for maskid in tqdm(range(100001, 110252)):
        # print('maskid =', maskid)
        df_patient = df[df['MaskID'] == maskid]
        if df_patient.shape[0] == 0: # no data for this patient
            continue
        sbp_fb_patient = get_feedback_patient(df_patient, 'sbp')
        hba1c_fb_patient = get_feedback_patient(df_patient, 'hba1c')
        cvdrisk_fb_patient = get_feedback_patient(df_patient, 'CVDRisk')
        bmi_fb_patient = get_feedback_patient(df_patient, 'BMI')
        TC_fb_patient = get_feedback_patient(df_patient, 'TC')
        hdl_fb_patient = get_feedback_patient(df_patient, 'hdl')
        sbp_fb.extend(sbp_fb_patient)
        hba1c_fb.extend(hba1c_fb_patient)
        cvdrisk_fb.extend(cvdrisk_fb_patient)
        bmi_fb.extend(bmi_fb_patient)
        TC_fb.extend(TC_fb_patient)
        hdl_fb.extend(hdl_fb_patient)
    
    # assert the length of feedback is the same as the original data
    assert len(sbp_fb) == df.shape[0]
    assert len(hba1c_fb) == df.shape[0]
    assert len(cvdrisk_fb) == df.shape[0]

    df['sbp_feedback'] = sbp_fb
    df['hba1c_feedback'] = hba1c_fb
    df['CVDRisk_feedback'] = cvdrisk_fb
    df['BMI_feedback'] = bmi_fb
    df['TC_feedback'] = TC_fb
    df['hdl_feedback'] = hdl_fb

    # overwrite the original csv
    df.to_csv(fn, index=False)

# add_feedback('data/ACCORD_BPMed_v2.csv')
# add_feedback('data/ACCORD_BGMed_v2.csv')
# add_feedback('data/ACCORD_BPBGMed_v2.csv')
add_feedback('data/ACCORD_BPClass_v2.csv')
# add_feedback('data/ACCORD_BGClass_v2.csv')
# add_feedback('data/ACCORD_BPBGClass_v2.csv')

fn = data/ACCORD_BPClass_v2.csv
(176906, 67)


100%|██████████| 10251/10251 [00:05<00:00, 2034.46it/s]


Discretize the feedbacks

In [10]:
# discretize the feedback
def discretize_feedback(fn):

    print('fn =', fn)
    df = pd.read_csv(fn)
    print(df.shape)

    # discretize sbp, hba1c, BMI, hdl, TC
    sbp_discrete = []
    hba1c_discrete = []
    BMI_discrete = []
    hdl_discrete = []
    TC_discrete = []
    for i in tqdm(range(df.shape[0])):
        row = df.iloc[i]
        sbp = row['sbp_feedback']
        hba1c = row['hba1c_feedback']
        BMI = row['BMI_feedback']
        hdl = row['hdl_feedback']
        TC = row['TC_feedback']

        if sbp < 120:
            sbp_discrete.append(0)
        elif sbp < 140:
            sbp_discrete.append(1)
        elif sbp < 160:
            sbp_discrete.append(2)
        else:
            sbp_discrete.append(3)

        if hba1c < 6:
            hba1c_discrete.append(0)
        elif hba1c < 6.5:
            hba1c_discrete.append(1)
        elif hba1c < 7:
            hba1c_discrete.append(2)
        elif hba1c < 7.5:
            hba1c_discrete.append(3)
        elif hba1c < 8:
            hba1c_discrete.append(4)
        elif hba1c < 8.5:
            hba1c_discrete.append(5)
        elif hba1c < 9:
            hba1c_discrete.append(6)
        else:
            hba1c_discrete.append(7)            

        if BMI < 18.5:
            BMI_discrete.append(0)
        elif BMI < 25:
            BMI_discrete.append(1)
        elif BMI < 30:
            BMI_discrete.append(2)
        else:
            BMI_discrete.append(3)

        if hdl < 40:
            hdl_discrete.append(0)
        elif hdl < 50:
            hdl_discrete.append(1)
        elif hdl < 60:
            hdl_discrete.append(2)
        else:
            hdl_discrete.append(3)

        if TC < 160:
            TC_discrete.append(0)
        elif TC < 200:
            TC_discrete.append(1)
        elif TC < 240:
            TC_discrete.append(2)
        else:
            TC_discrete.append(3)
    
    df['sbp_feedback_discrete'] = sbp_discrete
    df['hba1c_feedback_discrete'] = hba1c_discrete
    df['BMI_feedback_discrete'] = BMI_discrete
    df['hdl_feedback_discrete'] = hdl_discrete
    df['TC_feedback_discrete'] = TC_discrete

    # save to csv
    fn_out = fn
    # fn_out = fn.replace('.csv', '_v2.csv')
    df.to_csv(fn_out, index=False)


# discretize_feedback('data/ACCORD_BPMed_v2.csv')
# discretize_feedback('data/ACCORD_BGMed_v2.csv')
# discretize_feedback('data/ACCORD_BPBGMed_v2.csv')
discretize_feedback('data/ACCORD_BPClass_v2.csv')
# discretize_feedback('data/ACCORD_BGClass_v2.csv')  
# discretize_feedback('data/ACCORD_BPBGClass_v2.csv')

fn = data/ACCORD_BPClass_v2.csv
(176906, 68)


100%|██████████| 176906/176906 [00:26<00:00, 6698.58it/s]


One-hot encode BPMed and BGMed, BPClass and BGClass

In [27]:
# add prefix to 'none' values in med columns
def fix_none(fn, col_name, new_value):
    print('fn =', fn)
    df = pd.read_csv(fn)
    print(df.shape)

    # replace the none values in column 'bp_med_most5' with 'bpmed_none'
    df[col_name] = df[col_name].replace('none', new_value)
    df.to_csv(fn, index=False)

fix_none('data/ACCORD_BPMed_v2.csv', 'bp_med_most5', 'bpmed_none')
fix_none('data/ACCORD_BGMed_v2.csv', 'oral_gmed', 'bgmed_none')
fix_none('data/ACCORD_BPBGMed_v2.csv', 'bp_med_most5', 'bpmed_none')
fix_none('data/ACCORD_BPBGMed_v2.csv', 'oral_gmed', 'bgmed_none')

fix_none('data/ACCORD_BPClass_v2.csv', 'BPClass', 'bpclass_none')
fix_none('data/ACCORD_BGClass_v2.csv', 'BGClass', 'bgclass_none')
fix_none('data/ACCORD_BPBGClass_v2.csv', 'BPClass', 'bpclass_none')
fix_none('data/ACCORD_BPBGClass_v2.csv', 'BGClass', 'bgclass_none')

fn = data/ACCORD_BPMed_v2.csv
(178792, 51)
fn = data/ACCORD_BGMed_v2.csv
(351481, 51)
fn = data/ACCORD_BPBGMed_v2.csv
(167939, 51)
fn = data/ACCORD_BPBGMed_v2.csv
(167939, 51)
fn = data/ACCORD_BPClass_v2.csv
(177031, 51)
fn = data/ACCORD_BGClass_v2.csv
(290676, 52)
fn = data/ACCORD_BPBGClass_v2.csv
(139099, 52)
fn = data/ACCORD_BPBGClass_v2.csv
(139099, 52)


In [28]:
def onehot_encode(fn, col_name_list, names_list):
    print('fn =', fn)
    df = pd.read_csv(fn)
    print(df.shape)

    # build a dict with keys are names in names_list and values are empty lists
    dict_onehot = {}
    for name in names_list:
        dict_onehot[name] = []
    
    # loop through each row    
    for idx in tqdm(range(df.shape[0])):
        row = df.iloc[idx]
        med_set = set()
        for col_name in col_name_list:
            med = row[col_name]
            med_items = med.split('+')
            med_set.update(med_items)

        for name in names_list:
            if name in med_set:
                dict_onehot[name].append(1)
            else:
                dict_onehot[name].append(0)
    
    # add onehot columns to df
    for name in names_list:
        df[name] = dict_onehot[name]
        # check if the sum of each column is 0, check if any bpmed or bgmed has no occurrence, this means we messed up
        if df[name].sum() == 0:
            print("error: sum of column {} is 0".format(name))
            raise Exception("error: sum of column {} is 0".format(name))
    
    # overwrite the original csv
    fn_out = fn.replace('.csv', '_onehot.csv')
    df.to_csv(fn_out, index=False)


# 18 bp meds
bp_med_list = ['bpmed_none', 'metoprolol', 'hctz', 'lisinopril', 'chlorthalidone', 
            'valsartan', 'benazepril', 'furosemide', 'candesartan', 'ramipril', 
            'amlodipine', 'diltiazem', 'carvedilol', 'felodipine', 'terazosin', 
            'reserpine', 'triamterene', 'hydralazine']

# 11 bg meds    
bg_med_list = ['bgmed_none', 'metformin', 'rosiglitazone', 'glimepiride', 'repaglinide', 
            'acarbose', 'pioglitazone', 'exanatide', 'glyburide', 'sitagliptin', 
            'glipizide']

bpbg_med_list = bp_med_list + bg_med_list

# 10 bp classes
bp_class_list = ['bpclass_none', 'Diur', 'ACE', 'Beta-blocker', 'CCB', 
                'ARB', 'Alpha-Beta-blocker', 'Alpha-blocker', 'Sympath', 'Vasod']

# 6 bg classes
bg_class_list = ['bgclass_none', 'Bingu', 'Thiaz', 'Sulfon', 'Meglit', 'Alpha-gluc']

bpbg_class_list = bp_class_list + bg_class_list

onehot_encode('data/ACCORD_BPMed_v2.csv', ['bp_med_most5'], bp_med_list)
onehot_encode('data/ACCORD_BGMed_v2.csv', ['oral_gmed'], bg_med_list)
onehot_encode('data/ACCORD_BPBGMed_v2.csv', ['bp_med_most5','oral_gmed'], bp_med_list)

onehot_encode('data/ACCORD_BPClass_v2.csv', ['BPClass'], bp_class_list)
onehot_encode('data/ACCORD_BGClass_v2.csv', ['BGClass'], bg_class_list)
onehot_encode('data/ACCORD_BPBGClass_v2.csv', ['BPClass','BGClass'], bpbg_class_list)

fn = data/ACCORD_BPMed_v2.csv
(178792, 51)


100%|██████████| 178792/178792 [00:21<00:00, 8270.54it/s]


fn = data/ACCORD_BGMed_v2.csv
(351481, 51)


100%|██████████| 351481/351481 [00:43<00:00, 8091.73it/s]


fn = data/ACCORD_BPBGMed_v2.csv
(167939, 51)


...

100%|██████████| 177031/177031 [00:21<00:00, 8427.80it/s]


fn = data/ACCORD_BGClass_v2.csv
(290676, 52)


100%|██████████| 290676/290676 [00:34<00:00, 8470.78it/s]


fn = data/ACCORD_BPBGClass_v2.csv
(139099, 52)


100%|██████████| 139099/139099 [00:17<00:00, 7999.57it/s]


remove the suffix '_onehot' to fix v2 datasets names.

# Modify datasets for running RL algorithms

For ACCORD_BPClass_v2.csv, ACCORD_BGClass_v2.csv, ACCORD_BPBGClass_v2.csv:

1. remove patients/rows with missing edu_baseline, 125 rows with 3 patients in BPClass_v2.csv
2. add baseline_BMI column
3. one-hot enncode the raceclass into 2 category: race_whiteother, race_black
4. turn the CVDRisk_feedback into CVDRisk_feedback_binary

In [7]:
def update_dataset(fn):
    print('fn =', fn)
    df = pd.read_csv(fn)
    print(df.shape)

    # remove the rows with edu_baseline = -1, and count the number of rows
    df = df[df['edu_baseline'] != -1]
    print("after removing mising edu_baseline rows:", df.shape)

    baseline_BMI = []
    # loop through each row, calculate the baseline_BMI
    for idx in tqdm(range(df.shape[0])):
        row = df.iloc[idx]
        weight = row['wt_kg_baseline']
        height = row['ht_cm_baseline']
        bmi = weight / (height/100)**2
        baseline_BMI.append(bmi)
    
    # add a column 'baseline_BMI' to df
    df['baseline_BMI'] = baseline_BMI

    # overwrite the original csv
    df.to_csv(fn, index=False)

update_dataset('data/ACCORD_BPClass_v2.csv')

fn = data/ACCORD_BPClass_v2.csv
(177031, 61)
after removing mising edu_baseline rows: (176906, 61)


100%|██████████| 176906/176906 [00:25<00:00, 6834.65it/s]


In [8]:
update_dataset('data/ACCORD_BGClass_v2.csv')
update_dataset('data/ACCORD_BPBGClass_v2.csv')  

fn = data/ACCORD_BGClass_v2.csv
(290676, 58)
after removing mising edu_baseline rows: (290467, 58)


100%|██████████| 290467/290467 [00:41<00:00, 7033.61it/s]


fn = data/ACCORD_BPBGClass_v2.csv
(139099, 68)
after removing mising edu_baseline rows: (139005, 68)


100%|██████████| 139005/139005 [00:18<00:00, 7384.02it/s]


In [5]:
def onehot_encode_raceclass(fn):
    print('fn =', fn)
    df = pd.read_csv(fn)
    print(df.shape)

    race_whiteother = []
    race_black = []
    # loop through each row, calculate the baseline_BMI
    for idx in tqdm(range(df.shape[0])):
        row = df.iloc[idx]
        race = row['raceclass']
        if race == 'White' or race == 'Other' or race =='Hispanic':
            race_whiteother.append(1)
            race_black.append(0)
        elif race == 'Black':
            race_whiteother.append(0)
            race_black.append(1)
        else:
            print('race =', race)
            raise Exception("error: raceclass is not White, Black or Other")
            exit()
    
    df['race_whiteother'] = race_whiteother
    df['race_black'] = race_black

    # overwrite the original csv
    df.to_csv(fn, index=False)

onehot_encode_raceclass('data/ACCORD_BPClass_v2.csv')
onehot_encode_raceclass('data/ACCORD_BGClass_v2.csv')
onehot_encode_raceclass('data/ACCORD_BPBGClass_v2.csv')

fn = data/ACCORD_BPClass_v2.csv
(176906, 62)


100%|██████████| 176906/176906 [00:23<00:00, 7618.90it/s]


fn = data/ACCORD_BGClass_v2.csv
(290467, 59)


100%|██████████| 290467/290467 [00:39<00:00, 7285.03it/s]


fn = data/ACCORD_BPBGClass_v2.csv
(139005, 69)


100%|██████████| 139005/139005 [00:20<00:00, 6671.76it/s]


In [13]:
# turn the CVDRisk_feedback into a binary variable using threshold value of 0.5
def binarize_cvdrisk(fn):
    print('fn =', fn)
    df = pd.read_csv(fn)
    print(df.shape)

    CVDRisk_feedback_binary = []
    # loop through each row, calculate the baseline_BMI
    for idx in tqdm(range(df.shape[0])):
        row = df.iloc[idx]
        CVDRisk_feedback = row['CVDRisk_feedback']
        if CVDRisk_feedback >= 0.2: # here we use 0.2 as the threshold to make the binarized class balance, as only 3% data has CVDRisk_feedback >= 0.5
            CVDRisk_feedback_binary.append(1)
        else:
            CVDRisk_feedback_binary.append(0)
    
    df['CVDRisk_feedback_binary'] = CVDRisk_feedback_binary

    # print the mean of CVDRisk_feedback_binary
    print('mean of CVDRisk_feedback_binary =', df['CVDRisk_feedback_binary'].mean())
    print('median of CVDRisk_feedback_binary =', df['CVDRisk_feedback_binary'].median())
    print('std of CVDRisk_feedback_binary =', df['CVDRisk_feedback_binary'].std())

    # overwrite the original csv
    df.to_csv(fn, index=False)

binarize_cvdrisk('data/ACCORD_BPClass_v2.csv')
binarize_cvdrisk('data/ACCORD_BGClass_v2.csv')
binarize_cvdrisk('data/ACCORD_BPBGClass_v2.csv')

fn = data/ACCORD_BPClass_v2.csv
(176906, 65)


100%|██████████| 176906/176906 [00:24<00:00, 7152.17it/s]


mean of CVDRisk_feedback_binary = 0.49197879099634834
median of CVDRisk_feedback_binary = 0.0
std of CVDRisk_feedback_binary = 0.4999370690703593
fn = data/ACCORD_BGClass_v2.csv
(290467, 61)


100%|██████████| 290467/290467 [00:43<00:00, 6648.89it/s]


mean of CVDRisk_feedback_binary = 0.5505444680462841
median of CVDRisk_feedback_binary = 1.0
std of CVDRisk_feedback_binary = 0.497439552751152
fn = data/ACCORD_BPBGClass_v2.csv
(139005, 71)


100%|██████████| 139005/139005 [00:21<00:00, 6612.24it/s]


mean of CVDRisk_feedback_binary = 0.5183770367972375
median of CVDRisk_feedback_binary = 1.0
std of CVDRisk_feedback_binary = 0.49966396768047716


# Other

In [26]:
lst1 = ['a', 'b', 'c']
lst2 = ['d', 'e', 'f']
res = lst1 + lst2
print(res)

['a', 'b', 'c', 'd', 'e', 'f']


In [43]:
lst = ['a', 'b', '']
res = '+'.join(lst)
print(res)

a+b+


In [106]:
 med_visit = 'abc'
 med_list = med_visit.split('+')
 print(med_list)

['abc']


In [46]:
old = [-1,2,-1, 2, -1, 4, -1, -1]
print(old)
new = fill_with_adjacent_avg_list(old,0)
print(new)

[-1, 2, -1, 2, -1, 4, -1, -1]
[2, 2, 2.0, 2, 3.0, 4, 4, 4]


In [28]:
# create a dataframe with 1 column 'weight' with 5 rows value of 1,

wt = [1,1,None] 
df = pd.DataFrame(wt, columns=['weight'])
print(df['weight'].value_counts())
print(df['weight'].mean())


1.0    2
Name: weight, dtype: int64
1.0


In [ ]:
import pandas as pd
technologies = {
    'Courses':["Spark","PySpark","Python","pandas"],
    'Fee' :[20000,25000,22000,30000],
    'Duration':['30days','40days','35days','50days'],
              }
index_labels=['r1','r2','r3','r4']
df1 = pd.DataFrame(technologies,index=index_labels)

technologies2 = {
    'Courses':["Spark","Java","Python","Go"],
    'Discount':[2000,2300,1200,2000]
              }
index_labels2=['r1','r6','r3','r5']
df2 = pd.DataFrame(technologies2,index=index_labels2)

print(df1)
print(df2)

# pandas join two DataFrames
# df3=df1.join(df2, lsuffix="_left", rsuffix="_right", how='outer')
# print(df3)

# pandas.merge()
df3=pd.merge(df1,df2, how='outer')
print(df3)

# DataFrame.merge()
# df3=df1.merge(df2, how='outer')
# print(df3)

# Merge DataFrames by Columns
df3=pd.merge(df1,df2, on='Courses', how='outer')
print(df3)

# When column names are different
df3=pd.merge(df1,df2, left_on='Courses', right_on='Courses', how='outer')
print(df3)

# By using concat()
df3=pd.concat([df1,df2],axis=1,join='outer')
print(df3)


    Courses    Fee Duration
r1    Spark  20000   30days
r2  PySpark  25000   40days
r3   Python  22000   35days
r4   pandas  30000   50days
   Courses  Discount
r1   Spark      2000
r6    Java      2300
r3  Python      1200
r5      Go      2000
   Courses      Fee Duration  Discount
0    Spark  20000.0   30days    2000.0
1  PySpark  25000.0   40days       NaN
2   Python  22000.0   35days    1200.0
3   pandas  30000.0   50days       NaN
4     Java      NaN      NaN    2300.0
5       Go      NaN      NaN    2000.0
   Courses      Fee Duration  Discount
0    Spark  20000.0   30days    2000.0
1  PySpark  25000.0   40days       NaN
2   Python  22000.0   35days    1200.0
3   pandas  30000.0   50days       NaN
4     Java      NaN      NaN    2300.0
5       Go      NaN      NaN    2000.0
   Courses      Fee Duration  Discount
0    Spark  20000.0   30days    2000.0
1  PySpark  25000.0   40days       NaN
2   Python  22000.0   35days    1200.0
3   pandas  30000.0   50days       NaN
4     Java    